# Smart-G 3D demo notebook

This is an interactive document allowing to run Smart-G in 3D with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
from __future__ import absolute_import, division, print_function
import os, sys
#sys.path.insert(0, os.path.dirname(os.getcwd()))
sys.path.insert(0, '/home/did/RTC/SMART-G/')
from smartg.albedo import Albedo_speclib, Albedo_cst, Albedo_spectrum
from smartg.smartg import Smartg, Sensor, type_Cell,type_Profile, type_Sensor
from smartg.smartg import RoughSurface, LambSurface, FlatSurface, Environment
from smartg.atmosphere import AtmAFGL, AeroOPAC, CloudOPAC, diff1, read_phase, od2k, get_o2_abs, get_co2_abs
from smartg.water import IOP_1, IOP, IOP_profile
from smartg.geometry import BBox, Point, Vector, CommonFace, CommonVertices
from smartg.reptran import REPTRAN, reduce_reptran
from smartg.tools.tools import SpherIrr, Irr, reduce_Irr
from smartg.rrs import L2d_inv, is_odd
from smartg.bandset import spectral_grids
from smartg.lib3D import Get_3Dcells, Get_3Dcells_indices, Get_3Dcells_neighbours, locate_3Dregular_cells
from luts.luts import LUT, MLUT, Idx, merge, read_mlut
from smartg.tools.smartg_view import compare, plot_polar, spectrum , mdesc 
from smartg.tools.smartg_view import spectrum_view,transect_view,profile_view,phase_view,smartg_view,input_view
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")
from scipy.interpolate import interp1d
import scipy.constants as cst

# Kernel 2
from smartg.tools.modified_environ import modified_environ
import pycuda.driver as drv
import pycuda.tools

env_modif = {'CUDA_DEVICE': str(2)}
with modified_environ(**env_modif):
    import pycuda.autoinit
print(pycuda.autoinit.device.name())
from pycuda.compiler import SourceModule
from pycuda.gpuarray import to_gpu, zeros as gpuzeros
src_device = '/home/did/RTC/SMART-G/smartg/kernel2.cu'
source=open(src_device).read()
mod = SourceModule(source, nvcc='nvcc', no_extern_c=True)
reduce_absorption_gpu2 = mod.get_function("reduce_absorption_gpu2")
reduce_absorption_gpu  = mod.get_function("reduce_absorption_gpu")

In [ ]:
def locate_3D_cells(pro3D, xs, ys, zs):
    cond = np.greater_equal(pro3D['pmax_atm'].data[0,:][None,:], xs[:,None]) &\
        np.less(            pro3D['pmin_atm'].data[0,:][None,:], xs[:,None]) &\
        np.greater_equal(   pro3D['pmax_atm'].data[1,:][None,:], ys[:,None]) &\
        np.less(            pro3D['pmin_atm'].data[1,:][None,:], ys[:,None]) &\
        np.greater_equal(   pro3D['pmax_atm'].data[2,:][None,:], zs[:,None]) &\
        np.less(            pro3D['pmin_atm'].data[2,:][None,:], zs[:,None])
    
    return np.where(cond)[1]

# TOSCA 1D

## IPRT A5 case

In [ ]:
def read_3dmcpol_a5_pp(): 
    f_iprt = '../smartg/validation/IPRT/iprt_case_a5pp_3dmcpol.dat'
    th0    = 50.
    mu0    = np.cos(np.radians(th0))
    thv    = np.arange(0.0, 81.0, 1.0)
    phi    = np.array([0.0, 180.0])

    mSvRG = MLUT()
    mSvRG.add_axis('Azimuth angles', phi)
    mSvRG.add_axis('Zenith angles',  thv)
    
    f=open(f_iprt)
    for k, lev2 in enumerate(['down (0+)','up (TOA)']):
        I = np.full( (len(phi), len(thv) ), np.nan)
        Q = np.full( (len(phi), len(thv) ), np.nan)
        U = np.full( (len(phi), len(thv) ), np.nan)
        V = np.full( (len(phi), len(thv) ), np.nan)
        Is = np.full( (len(phi), len(thv) ), np.nan)
        Qs = np.full( (len(phi), len(thv) ), np.nan)
        Us = np.full( (len(phi), len(thv) ), np.nan)
        Vs = np.full( (len(phi), len(thv) ), np.nan)
        for iphi in range(len(phi)): 
            for imu in range(len(thv)):
                tmp = f.readline().split()
                if (k==0):
                    I[iphi,imu] = float(tmp[6])   * np.pi/mu0
                    Q[iphi,imu] = float(tmp[7])   * np.pi/mu0
                    U[iphi,imu] = float(tmp[8])   * np.pi/mu0
                    V[iphi,imu] = float(tmp[9])   * np.pi/mu0
                    Is[iphi,imu] = float(tmp[10]) * np.pi/mu0
                    Qs[iphi,imu] = float(tmp[11]) * np.pi/mu0
                    Us[iphi,imu] = float(tmp[12]) * np.pi/mu0
                    Vs[iphi,imu] = float(tmp[13]) * np.pi/mu0
                else:
                    I[iphi,-imu-1] = float(tmp[6])   * np.pi/mu0
                    Q[iphi,-imu-1] = float(tmp[7])   * np.pi/mu0
                    U[iphi,-imu-1] = float(tmp[8])   * np.pi/mu0
                    V[iphi,-imu-1] = float(tmp[9])   * np.pi/mu0
                    Is[iphi,-imu-1] = float(tmp[10]) * np.pi/mu0
                    Qs[iphi,-imu-1] = float(tmp[11]) * np.pi/mu0
                    Us[iphi,-imu-1] = float(tmp[12]) * np.pi/mu0
                    Vs[iphi,-imu-1] = float(tmp[13]) * np.pi/mu0

        mSvRG.add_dataset('I_'+lev2, I, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('Q_'+lev2, Q, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('U_'+lev2, U, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('V_'+lev2, V, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('N_'+lev2, np.zeros_like(I),   ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('I_stdev_'+lev2, Is, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('Q_stdev_'+lev2, Qs, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('U_stdev_'+lev2, Us, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('V_stdev_'+lev2, Vs, ["Azimuth angles","Zenith angles"])

    return mSvRG 

def read_3dmcpol_c2_pp(iset=1, icase=4):
    
    f_iprt = '../smartg/validation/IPRT/iprt_case_C2_3DMCPOL.dat'
    th0    = 40.
    mu0    = np.cos(np.radians(th0))
    thv    = np.array([0.])
    phi    = np.array([0.])
    si     = np.arange(4900)

    mSvRG = MLUT()
    mSvRG.add_axis('Zenith angles',  thv)
    mSvRG.add_axis('sensor index' ,  si)
    
    f=open(f_iprt)
    lev2 = 'up (TOA)'

    I  = np.full( (len(si), len(thv) ), np.nan)
    Q  = np.full( (len(si), len(thv) ), np.nan)
    U  = np.full( (len(si), len(thv) ), np.nan)
    V  = np.full( (len(si), len(thv) ), np.nan)
    Is = np.full( (len(si), len(thv) ), np.nan)
    Qs = np.full( (len(si), len(thv) ), np.nan)
    Us = np.full( (len(si), len(thv) ), np.nan)
    Vs = np.full( (len(si), len(thv) ), np.nan)
    
    imu=0

    for k in range(3 + (4900*9*iset) + (4900*icase)):
        f.readline()
    for i in range(4900):
        tmp = f.readline().split()
        I[i,imu]  = float(tmp[7])  * np.pi/mu0
        Q[i,imu]  = float(tmp[8])  * np.pi/mu0
        U[i,imu]  = float(tmp[9])  * np.pi/mu0
        V[i,imu]  = float(tmp[10])  * np.pi/mu0
        Is[i,imu] = float(tmp[11]) * np.pi/mu0
        Qs[i,imu] = float(tmp[12]) * np.pi/mu0
        Us[i,imu] = float(tmp[13]) * np.pi/mu0
        Vs[i,imu] = float(tmp[14]) * np.pi/mu0
        
    mSvRG.add_dataset('I_'+lev2, I, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('Q_'+lev2, Q, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('U_'+lev2, U, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('V_'+lev2, V, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('N_'+lev2, np.zeros_like(I),   ["sensor index","Zenith angles"])
    mSvRG.add_dataset('I_stdev_'+lev2, Is, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('Q_stdev_'+lev2, Qs, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('U_stdev_'+lev2, Us, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('V_stdev_'+lev2, Vs, ["sensor index","Zenith angles"])

    return mSvRG 

In [ ]:
# Read 3DMCPOL from IPRT website
mcpol_pp=read_3dmcpol_a5_pp()

# Cloud Phase matrix
cld_phase = read_mlut('/home/did/RTC/SMART-G/smartg/validation/watercloud_IPRT_A5.mie.nc')
ntheta = cld_phase['ntheta'][0,0,0]
wl     = cld_phase['wavelen'].data*1e3
wl=[350.]
l=[]
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() + cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() - cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,2,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,3,:ntheta].ravel())
# Normalization to 2
p=cld_phase['phase'][:,:,0,:ntheta].ravel()
theta=cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]
mu= np.cos(np.radians(theta))
Norm = np.trapz(p,-mu)
data = np.array(l) 
print ('phase function norm: ', Norm)
#data = np.array(l) * (2./Norm)
# 2) store data in a LUT object with 4 dimensions, the z dimension is restricted to one level of altitude 0
pha_cld = LUT(data[None, None, :, ::-1], names=['wav_phase', 'z_phase', 'stk', 'theta_atm'],
          axes=[wl, np.array([0.]), None, cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]]
         )

# Layering and angles
th0  = 50.
COT  = 5. # at 800 nm
cld  = CloudOPAC('wc.sol', 3., 0.5, 10., COT, wl, phase=pha_cld)
NL   = 1
grid = np.linspace(100, 0., num=NL+1)
atm  = AtmAFGL('afglt', O3=0., comp=[cld], tauR=1e-20, NO2=False, grid=grid)
le   = {'phi_deg':np.linspace(0.,180., num=2), 'th_deg':np.linspace(0., 80., num=81)}

### Principal plane

In [ ]:
%%time
ma5_1e6 = Smartg(double=True, alt_pp=True).run(wl=800., atm=atm, le=le, 
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=3, THVDEG=th0,
                                           stdev=True)

In [ ]:
# 3DMCPOL 1e8 photons 21 hours 25 min for one field (TOA)
T = (21*3600+ 25*60)
# 3DMCPOL 1e6 photons 20 mins 06 sec  for one field (TOA)
#T = 20*60 + 6
print ('3DMCPOL 1e8: %.1f (s)'%T)
print(pycuda.autoinit.device.name()+' : %.2f (s)'%(float(ma5_1e6.attrs['kernel time (s)'])))

In [ ]:
_=compare(ma5_1e6, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.01, 0.001, 0.001, 0.5], ermax=[2,5,20,5])

In [ ]:
_=compare(ma5_1e6, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.01, 0.001, 0.001, 0.5], ermax=[2,5,20,5])

In [ ]:
_=compare(ma5_1e8, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.01, 0.001, 0.001, 0.5], ermax=[2,5,20,5])

In [ ]:
_=compare(ma5_1e8, mcpol_pp,same_azimuth_convention=True, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.05, 0.001, 10], vmin=[-1., -0.05, -0.001, 0], field='down (0+)', logI=True,
        emax=[0.5, 0.001, 0.001, 0.5], ermax=[5,5,20,5])

In [ ]:
%%time
ma5_1e6 = Smartg(double=True, alt_pp=False).run(wl=800., atm=atm, le=le, 
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=3, THVDEG=th0,
                                           stdev=True)

In [ ]:
_=compare(ma5_1e6, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.1, 0.01, 0.01, 1], ermax=[10,10,10,10])

In [ ]:
%%time
ma5_alt_1e6 = Smartg(double=True, alt_pp=True).run(wl=800., atm=atm, le=le,
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=3, THVDEG=th0,
                                            stdev=True)

In [ ]:
_=compare(ma5_alt_1e6, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.1, 0.01, 0.01, 1], ermax=[10,10,10,10])

## IPRT B1 case

In [ ]:
NL   = 30
grid = np.linspace(30, 0., num=NL+1)
atm  = AtmAFGL('afglt', O3=0., NO2=False, grid=grid)
le   = {'phi_deg':np.linspace(0.,180., num=37), 'th_deg':np.linspace(0., 85., num=19)}
th0  = 50.

In [ ]:
%%time
m1DA_B1_1e6 = Smartg(double=True, alt_pp=True, alis=True).run(wl=450., atm=atm, le=le, NBLOOP=1e5, NF=1e3,
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=0, THVDEG=th0,
                                           stdev=False, alis_options={'nlow':-1})

In [ ]:
%%time
mfast1DA_B1_1e6 = Smartg(double=True, alt_pp=False, alis=True).run(wl=450., atm=atm, le=le, NBLOOP=1e5, NF=1e3,
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=0, THVDEG=th0,
                                           stdev=False, alis_options={'nlow':-1})

In [ ]:
%%time
m1D_B1_1e6 = Smartg(double=True, alt_pp=True, alis=False).run(wl=450., atm=atm, le=le, NBLOOP=1e5, NF=1e3,
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=0, THVDEG=th0,
                                           stdev=False, alis_options={'nlow':-1})

In [ ]:
%%time
mfast1D_B1_1e6 = Smartg(double=True, alt_pp=False, alis=False).run(wl=450., atm=atm, le=le, NBLOOP=1e5, NF=1e3,
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=0, THVDEG=th0,
                                           stdev=False, alis_options={'nlow':-1})

In [ ]:
_=smartg_view(mfast1DA_B1_1e6)

# TOSCA 3D

## Bounding Box

![Image of Bounding Box](https://static.packt-cdn.com/products/9781787123663/graphics/B05887_7_5.jpg)

## Cube Mapping

![Image of Cube Mapping](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Cube_map.svg/599px-Cube_map.svg.png)

## IPRT B1

In [ ]:
# B1 IPRT data
Nz      = 30
zgrid   = np.linspace(0, 30., num=Nz+1)
Nx, Ny  = 1, 1
Nl      = Nz + 1                  # number of levels in Z
Dx, Dy  = 1e6, 1e6           # horizontal grid interval in km
assert Nl==zgrid.size

#zgrid = np.concatenate([zgrid, np.array([120.])])
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Dx=Dx, Dy=Dy, z=zgrid, SAT_ALTITUDE=110, periodic=False, HORIZ_EXTENT_LENGTH=0)
Ncell   = NX*NY*NZ
znew    = zgrid[::-1]
atm     = AtmAFGL('afglt', O3=0., NO2=False, grid=znew)

# Sun position
SAA    = 0.
SZA    = 50.
POSX   = 0.
POSY   = 0.
POSZ   = 29.99
# LE directions
le   = {'phi_deg':np.linspace(0.,180., num=37), 'th_deg':np.linspace(0., 85., num=19)}
# cell index
ICELL = locate_3Dregular_cells(xgrid, ygrid,zgrid, POSX, POSY, POSZ)
# sensor object
sensor = Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=180.-SZA, PHDEG=180.-SAA, LOC='ATMOS', ICELL=ICELL)

#Rayleigh
wls         = [450.]
sca_ray     = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(wls), 'OD_r')
sca_ray3D   = np.concatenate([sca_ray], axis=1)
sca_ray3D   = sca_ray

In [ ]:
## Mapping optical properties and cells
# 1 clear profile + number of cloud cells opt. prop. (here 1 per cell)
Nopt        = NZ + 1 
iopt        = np.zeros(Ncell, dtype=np.int32)
iabs        = np.zeros_like(iopt)
iopt[:]     = np.arange(Nopt)[NZ-idz] # Scattering depending on Z for clear atmosphere (Rayleigh)
iabs[:]     = np.arange(Nopt)[NZ-idz] # Absorption depending on Z only

atm_arr = np.zeros((Nopt,9))
#atm_arr[:,0] = np.arange(Nopt)[::-1]
atm_arr[:,0] = zgrid
np.savetxt('./tmp.dat', atm_arr)
Nabs        = iabs.max().astype(np.int32)

### profiles computations
#atm3D = AtmAFGL('./tmp.dat', US=False,
atm3D = AtmAFGL('afglt', US=False,
                    grid        = zgrid,
                    prof_ray    = sca_ray3D,
                    cells       = (iopt, iabs, pmin, pmax, neigh)
                   )
pro3D  = atm3D.calc(wls)

In [ ]:
%%time
NB     = 1e6
m3D_B1_1e6    = Smartg(opt3D=True).run(wl=wls, NBPHOTONS=NB, NBLOOP=1e5, atm=pro3D.sub({'wavelength':np.arange(1)}), 
                 sensor=sensor, le=le, NF=1e3).sub({'wavelength':np.arange(1)})

In [ ]:
NB      = 1e6
m3DA_B1_1e6    = Smartg(opt3D=True, alis=True).run(wl=wls, NBPHOTONS=NB, NBLOOP=1e5, atm=pro3D.sub({'wavelength':np.arange(1)}), 
                 sensor=sensor, le=le, NF=1e3, alis_options={'nlow':-1}).sub({'wavelength':np.arange(1)})

In [ ]:
# 3DMCPOL 1e6 photons 8 mins 03 sec  for one field (TOA)
T = 8*60 + 3
print ('3DMCPOL: %.1f'%T)
print('%.2f'%(float(m3DA_B1_1e6.attrs['kernel time (s)'])))
print('%.2f'%(float(m3D_B1_1e6.attrs['kernel time (s)'])))
print('%.2f'%(float(m1DA_B1_1e6.attrs['kernel time (s)'])))
print('%.2f'%(float(m1D_B1_1e6.attrs['kernel time (s)'])))
print('%.2f'%(float(mfast1DA_B1_1e6.attrs['kernel time (s)'])))
print('%.2f'%(float(mfast1D_B1_1e6.attrs['kernel time (s)'])))

## IPRT C2

In [ ]:
# Cloud Phase matrix
cld_phase = read_mlut('/home/did/RTC/SMART-G/smartg/validation/IPRT/watercloud.mie.nc')
ntheta = cld_phase['ntheta'][0,0,0]
wl     = [670.]
l      = []
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() + cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() - cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,2,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,3,:ntheta].ravel())
# Normalization to 2
p      = cld_phase['phase'][:,:,0,:ntheta].ravel()
theta  = cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]
mu     = np.cos(np.radians(theta))
NORM   = np.trapz(p,-mu)
data   = np.array(l) 
print ('phase function norm: ', NORM)
#data = np.array(l) * (2./Norm)
# 2) store data in a LUT object with 4 dimensions, the z dimension is restricted to one level of altitude 0
pha_cld = LUT(data[:, ::-1], names=['stk', 'theta_atm'],
          axes=[None, cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]]
         )

In [ ]:
# vertical grid in descending order as usual for SmartG 1D
znew = np.array([100., 99., 20., 5., 3., 2., 0.])
Nx=1
Ny=1
Dx=1
Dy=1
# vertical grid in ascending order for 3D grid
zgrid = znew[::-1]

# grid building
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ

# cloud cells
idx_cloud  = [1]
idy_cloud  = [1]
idz_cloud  = [1]
cloud_cell = np.ravel_multi_index((idx_cloud, idy_cloud, idz_cloud), 
                                  dims=(NX,NY,NZ), order='C')

In [ ]:
wls = [362.]
ODR = 0.5
#Rayleigh
sca_ray     = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew, tauR=0.5).calc(wls), 'OD_r')
#Cloud
sca_cld     = [10.]

# Concatenation
sca_cld3D = np.concatenate([np.zeros_like(sca_ray), np.stack([sca_cld]*cloud_cell.size, axis=1)], axis=1)
sca_ray3D = np.concatenate([sca_ray, sca_ray[:,NZ-idz[cloud_cell]]], axis=1)
ssa_cld3D = np.ones_like(sca_cld3D)
ipha3D    = np.zeros_like(sca_cld3D, dtype=int32)

## Mapping optical propoerties and cells
# 1 clear profile + number of cloud cells opt. prop. (here 1 per cell)
Nopt       = NZ + 1 + cloud_cell.size
iopt       = np.zeros(Ncell, dtype=int32)
iabs       = np.zeros_like(iopt)
iopt[:]    = np.arange(Nopt)[NZ-idz] # Scattering depending on Z for clear atmosphere (Rayleigh)
iabs[:]    = np.arange(Nopt)[NZ-idz] # Absorption depending on Z only
iopt[cloud_cell] = NZ + 1 + np.arange(cloud_cell.size)  # Scattering depending on cloud cell

atm3D = AtmAFGL('afglt',
                    grid        = np.arange(Nopt),
                    prof_ray    = sca_ray3D,
                    prof_aer    = (sca_cld3D, ssa_cld3D),
                    prof_phases = (ipha3D, [pha_cld]),
                    cells       = (iopt, iabs, pmin, pmax, neigh)
                   )
pro3D  = atm3D.calc(wls)

In [ ]:
# Sun position
SAA    = 180.
SZA    = 40.
# Sensors position
Npix   = 70
POSZ   = 55.0001
POSX   = 3.4999
POSY   = 3.4999
THDEG  = 180.
PHDEG  = 0.
# pixels centers on the ground
x0    = np.linspace(-POSX, POSX, num=Npix) # central domain (cumulus) boundaries
y0    = np.linspace(-POSY, POSY, num=Npix)
z0    = np.linspace( POSZ,   5 , num=Npix)
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid, zgrid, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,POSZ,ICELL in zip(xx.ravel(), yy.ravel(), zz.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

In [ ]:
%%time
S3D    = Smartg(opt3D=True,  alt_pp=True, alis=False) # 3D
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
NB     = len(sensors) * 1e5
surf   = LambSurface(ALB=Albedo_cst(0.2))
m3D    = S3D.run(wl=wl, DEPO=0.2, NBPHOTONS=NB, atm=pro3D, sensor=sensors, le=le, surf=surf, NBLOOP=1e7, NF=1e4)

In [ ]:
for k in range(1):
    fig,ax=subplots(2,2)
    fig.set_size_inches((14,10))
    I = m3D['I_up (TOA)'][:,0,0].reshape((Npix,Npix))
    Q = m3D['Q_up (TOA)'][:,0,0].reshape((Npix,Npix))
    U = m3D['U_up (TOA)'][:,0,0].reshape((Npix,Npix))
    V = m3D['V_up (TOA)'][:,0,0].reshape((Npix,Npix))
    P1=ax[0,0].imshow(I, vmin=0., vmax=0.6, cmap=cm.jet)
    colorbar(P1, ax=ax[0,0], shrink=0.8)
    P2=ax[0,1].imshow(Q, vmin=-0.1, vmax=0.1, cmap=cm.RdBu_r)
    colorbar(P2, ax=ax[0,1], shrink=0.8)
    P3=ax[1,0].imshow(-U, vmin=-0.0005, vmax=0.0005, cmap=cm.RdBu_r)
    colorbar(P3, ax=ax[1,0], shrink=0.8)
    P4=ax[1,1].imshow(V, vmin=-0.0001, vmax=0.0001, cmap=cm.RdBu_r)
    colorbar(P4, ax=ax[1,1], shrink=0.8)
    #P4=ax[1,1].imshow(np.sqrt(Q**2+U**2)/I*100, vmin=0, vmax=40, cmap=cm.RdBu_r)
    #colorbar(P4, ax=ax[1,1], shrink=0.8)

In [ ]:
# 3DMCPOL results
mcpol_c2  = read_3dmcpol_c2_pp(iset=1).dropaxis('Zenith angles').describe()
for k in range(1):
    fig,ax=subplots(2,2)
    fig.set_size_inches((14,10))
    I = mcpol_c2['I_up (TOA)'][:].reshape((Npix,Npix))
    Q = mcpol_c2['Q_up (TOA)'][:].reshape((Npix,Npix))
    U = mcpol_c2['U_up (TOA)'][:].reshape((Npix,Npix))
    V = mcpol_c2['V_up (TOA)'][:].reshape((Npix,Npix))
    P1=ax[0,0].imshow(I.T, vmin=0., vmax=0.6, cmap=cm.jet)
    colorbar(P1, ax=ax[0,0], shrink=0.8)
    P2=ax[0,1].imshow(Q.T, vmin=-0.1, vmax=0.1, cmap=cm.RdBu_r)
    colorbar(P2, ax=ax[0,1], shrink=0.8)
    P3=ax[1,0].imshow(-U.T, vmin=-0.0005, vmax=0.0005, cmap=cm.RdBu_r)
    colorbar(P3, ax=ax[1,0], shrink=0.8)
    P4=ax[1,1].imshow(V.T, vmin=-0.0001, vmax=0.0001, cmap=cm.RdBu_r)
    colorbar(P4, ax=ax[1,1], shrink=0.8)
    #P4=ax[1,1].imshow(np.sqrt(Q**2+U**2)/I*100, vmin=0, vmax=40, cmap=cm.RdBu_r)
    #colorbar(P4, ax=ax[1,1], shrink=0.8)

## LES cloud POLDER

### Scene

In [ ]:
# C3 IPRT Cumulus data
cumulus_f='/home/did/RTC/SMART-G/smartg/validation/IPRT/cumulus.dat'
f       = open(cumulus_f, 'r')
f.readline(); 
l       = f.readline().split()
Nx, Ny, Nz, _ = list(map(int, l)) # number of cumulus cells in X, Y, X
Nl      = Nz + 1                  # number of levels in Z
l       = f.readline()
g_info  = list(map(float, l.split()))
Dx, Dy  = g_info[0:2]            # horizontal grid interval in km
#Dx*=10. ; Dy*=10.
zgrid   = np.array(g_info[2:])     # vertical grid in km
assert Nl==zgrid.size

zgrid = np.concatenate([zgrid, np.array([120.])])
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Dx=Dx, Dy=Dy, z=zgrid, SAT_ALTITUDE=120, periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell = NX*NY*NZ
znew  = zgrid[::-1]

In [ ]:
SENTINEL_POLDER = REPTRAN('reptran_solar_parasol')
ibands = SENTINEL_POLDER.to_smartg(lmin=880, lmax=990) # select POLDER band water vapour
wl_ref = 670.
w0 = []
for b in ibands.l:   w0.append(b.w)
#alternative reff = 2 micron phase function
cld = CloudOPAC('wc.sol', 1., 2., 3., 1., wl_ref)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl_ref)['phase_atm'].sub()[0,:,:]
# white Cumulus C3 LES cloud ksca
cumulus     = np.loadtxt(cumulus_f, skiprows=3, dtype=np.float32)
sca_cld     = np.stack([cumulus[:,3]]*len(ibands.l), axis=0) # km-1

#Rayleigh
sca_ray     = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(ibands.l), 'OD_r')
abs_gas     = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(ibands.l), 'OD_g')

# indexing of C3 cells into SMART-G grid
cloud_geo   = cumulus[:,:3].astype(int32)-1
cloud_cell  = np.ravel_multi_index((cloud_geo[:,0], cloud_geo[:,1], cloud_geo[:,2]),
                    dims=(NX, NY, NZ))

# Concatenation
sca_cld3D   = np.concatenate([np.zeros_like(sca_ray), sca_cld], axis=1)
sca_ray3D   = np.concatenate([sca_ray, sca_ray[:,NZ-idz[cloud_cell]]], axis=1)
abs_gas3D   = np.concatenate([abs_gas, abs_gas[:,NZ-idz[cloud_cell]]], axis=1)
ssa_cld3D   = np.ones_like(sca_cld3D)
ipha3D      = np.zeros_like(sca_cld3D, dtype=int32)

## Mapping optical propoerties and cells
# 1 clear profile + number of cloud cells opt. prop. (here 1 per cell)
Nopt        = NZ + 1 + cloud_cell.size
iopt        = np.zeros(Ncell, dtype=int32)
iabs        = np.zeros_like(iopt)
iopt[:]     = np.arange(Nopt)[NZ-idz] # Scattering depending on Z for clear atmosphere (Rayleigh)
iabs[:]     = np.arange(Nopt)[NZ-idz] # Absorption depending on Z only
iopt[cloud_cell] = NZ + 1 + np.arange(cloud_cell.size)  # Scattering depending on cloud cell

atm_arr = np.zeros((Nopt,9))
atm_arr[:,0] = np.arange(Nopt)[::-1]
np.savetxt('./tmp.dat', atm_arr)
Nabs        = iabs.max().astype(np.int32)

### profiles computations
atm3D = AtmAFGL('./tmp.dat', US=False,
                    grid        = np.arange(Nopt),
                    prof_abs    = abs_gas3D,
                    prof_ray    = sca_ray3D,
                    prof_aer    = (sca_cld3D, ssa_cld3D),
                    prof_phases = (ipha3D, [pha_cld]),
                    cells       = (iopt, iabs, pmin, pmax, neigh)
                   )
pro3D  = atm3D.calc(ibands.l)

# Ground
ALB     = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.grass.green.solid.gras.spectrum.txt')
surf    = LambSurface(ALB=Albedo_cst(ALB.get(w0)))

### View 1

In [ ]:
# Sun position
SAA    = 180.
SZA    = 40.
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
# Sensors position
Npix   = 10
POSZ   = 30.
THDEG  = 180.
PHDEG  = 180.
# pixels centers on the ground
x0     = np.linspace(-Nx*Dx/2., Nx*Dx/2, num=Npix) # central domain (cumulus) boundaries
y0     = np.linspace(-Ny*Dy/2., Ny*Dy/2, num=Npix)
#
xx,yy  = np.meshgrid(x0, y0)
zz     = np.zeros_like(xx) + POSZ

# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid,zgrid, xx.ravel(), yy.ravel(), zz.ravel())
# sensors objects
sensors1=[]
for POSX,POSY,ICELL in zip(xx.ravel(), yy.ravel(), icells):
    sensors1.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

In [ ]:
%%time
NB     = len(sensors1) * 1e2
m3D1   = reduce_reptran(Smartg(opt3D=True).run(ibands.l, NBPHOTONS=NB, NBLOOP=1e4, atm=pro3D, 
                 sensor=sensors1, le=le, surf=surf, NF=1e4), ibands, use_solar=True).describe()

In [ ]:
rcParams.update({'font.size':18})
f,ax = subplots(1,1)
f.set_size_inches(8,8)
II   = m3D1['I_up (TOA)'][:,:,0,0].reshape(Npix,Npix,-1)
im   = II[-1:1:-1,1:-1:1,0]
img  = ax.imshow(im, cmap=cm.Blues_r, vmax=2000, extent=[x0.min(),x0.max(),y0.min(),y0.max()])
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
cbar = f.colorbar(img, shrink=0.9, orientation='vertical')
cbar.ax.set_xlabel(mdesc('I_up (TOA)')+r'$ (mW.m^{-2}.sr^{-1}.nm^{-1})$')

### View 2

In [ ]:
from smartg.geometry import Point, Vector
# Sun position
SAA    = 180.
SZA    = 40.
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
# Sensors position
Npix   = 10
POSX   = 0.05
POSY   = 0.05
POSZ   = 30.
pS     = Point(POSX,POSY,POSZ) # Camera position
ICELL  = locate_3Dregular_cells(xgrid, ygrid, zgrid, POSX, POSY, POSZ)
# pixels centers on the ground
x0     = np.linspace(-Nx*Dx/2., Nx*Dx/2, num=Npix) # central domain (cumulus) boundaries
y0     = np.linspace(-Ny*Dy/2., Ny*Dy/2, num=Npix)
xx,yy  = np.meshgrid(x0, y0)
sensors2= []
for x,y in zip(xx.ravel(), yy.ravel()):   
    pF  = Point(x, y, 0.)
    vS  = Vector(pF-pS) # Sensor pointing vector
    sensors2.append(Sensor(POSX=pS.x, POSY=pS.y, POSZ=pS.z, LOC='ATMOS', FOV=0., TYPE=0, V=vS, ICELL=ICELL))

In [ ]:
%%time
NB     = len(sensors2) * 1e2
m3D2   = reduce_reptran(Smartg(opt3D=True, verbose_photon=False).run(ibands.l, NBPHOTONS=NB, NBLOOP=1e2, atm=pro3D, 
                 sensor=sensors2, le=le, surf=surf, NF=1e4), ibands, use_solar=True).describe()

In [ ]:
rcParams.update({'font.size':18})
f,ax = subplots(1,1)
f.set_size_inches(8,8)
II   = m3D2['I_up (TOA)'][:,:,0,0].reshape(Npix,Npix,-1)
im   = II[-1:1:-1,1:-1:1,0]
img  = ax.imshow(im, cmap=cm.Blues_r, vmax=2000, extent=[x0.min(),x0.max(),y0.min(),y0.max()])
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
cbar = f.colorbar(img, shrink=0.9, orientation='vertical')
cbar.ax.set_xlabel(mdesc('I_up (TOA)')+r'$ (mW.m^{-2}.sr^{-1}.nm^{-1})$')

In [ ]:
for sen in sensors2:
    print(sen)

# LPS2019

In [ ]:
import ephem

def Get_SunEarth(obs):
    """
    Get Earth-Sun distance correction
    """
    date = str(obs.date).split()[0]
    jday = datetime.datetime.strptime(date, '%Y/%m/%d').timetuple().tm_yday
    SE_corr_c1 = 1.00011
    SE_corr_c2 = 0.034221
    SE_corr_c3 = 0.00128
    SE_corr_c4 = 0.000719
    SE_corr_c5 = 0.000077

    d_qo = 2.0 * np.arccos(-1.) * (jday + 0.35) / 365.
    d_corr = SE_corr_c1 +                       \
             SE_corr_c2 * np.cos(d_qo) +           \
             SE_corr_c3 * np.sin(d_qo) +           \
             SE_corr_c4 * np.cos(d_qo * 2.) +      \
             SE_corr_c5 * np.sin(d_qo * 2.)
    dist = 1.0 / np.sqrt(d_corr)

    return dist * dist


lon = 0.
lat = 35.

dates= ['2015/6/21','2015/9/20']

dt  = 75./(24*60) # time interval in fraction of day, here 30 minutes

obs      = ephem.Observer()
obs.lon  = str(lon)
obs.lat  = str(lat)
obs.pressure=0.
obs.horizon = '0:34'
obs.date = dates[1]
sun      = ephem.Sun()
drise    = obs.next_rising (sun, start=obs.date)
dset     = obs.next_setting(sun, start=obs.date)
dnoon    = obs.next_transit(sun, start=obs.date)
#halfday  = dnoon - drise # computation from noon to sunrise
fullday  = dset  - drise # computation from noon to sunrise

#t        = np.linspace(0.001, halfday, num=halfday/dt, endpoint=True)
t        = np.linspace(0.00, fullday, num=fullday/dt, endpoint=False)

dates=[]
th0=[]
sed=[]
ph0 = []
for i,dt in enumerate(t):
    #da       = dnoon - dt
    da       = dset  - dt
    obs.date = da
    sun.compute(obs)
    th0.append( 90.-float(sun.alt)*180/np.pi)
    ph0.append(float(sun.az)*180./np.pi)
    dates.append(str(obs.date))
    sed.append(Get_SunEarth(obs))
th0=np.array(th0)
ph0=np.array(ph0)
mus = cos(th0*np.pi/180)

# Solar geometries
le={'th_deg':th0, 'phi_deg':ph0, 'zip':True}

# Profile to Cell

## Simple atmosphere

In [ ]:
# vertical grid in descending order as usual for SmartG 1D
znew = np.linspace(50., 0. , num=26)
#znew = np.array([100., 99., 20., 5., 3., 2., 0.])
Nx=1
Ny=1
Dx=1e5
Dy=1e5
# vertical grid in ascending order for 3D grid
zgrid = znew[::-1]

# grid building
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=0)
Ncell   = NX*NY*NZ

# cells optical properties
iopt           = np.zeros(NX*NY*NZ, dtype= int32)
# properties depend only on z
iatm           = np.arange(NZ)
iopt[:]        = iatm[idz]+1

atm3D_clr = AtmAFGL('afglt', O3=0., NO2=False,
                    grid        = zgrid,
                    cells       = (iopt, pmin, pmax, neigh)
                   )
atm_clr   = AtmAFGL('afglt', O3=0., NO2=False,
                    grid        = zgrid[::-1]
                   )
wl = np.linspace(400., 420., num=5)
wl = [400.]

pro3D_clr = atm3D_clr.calc(wl)
pro_clr   = atm_clr.calc(wl)

In [ ]:
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax

In [ ]:
# Sensors position
POSX = 0.005
POSY = 0.005
POSZ = 110.00001
P    = Point(z=POSZ)
icells = locate_3D_cells(pro3D_clr, np.array([POSX]), np.array([POSY]), np.array([POSZ]))
ICELL = icells[0]
print(ICELL)
# pixels centers on the ground
x0    = xgrid # central domain (cumulus) boundaries
y0    = ygrid
# centers
xx,yy = np.meshgrid(np.diff(x0)/2. + x0[:-1], np.diff(y0)/2. + y0[:-1])
# view angles
PHDEG = np.degrees(np.arctan2(xx, yy))
THDEG = 180-np.degrees(np.arctan2(np.sqrt(xx**2+yy**2), POSZ))

sensors=[]
for th, ph in zip(THDEG.ravel(), PHDEG.ravel()):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=th, PHDEG=ph, LOC='ATMOS', ICELL=ICELL))

In [ ]:
%%time
S3D    = Smartg(opt3D=True,  alt_pp=True) # 3D
S      = Smartg(opt3D=False, alt_pp=True) # 3D
le     = {'th_deg':np.linspace(0, 80., num=1), 'phi_deg':np.linspace(0., 210., num=1)}

NB     = 1e7
surf   = RoughSurface()
surf   = None
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_clr, sensor=sensors, le=None, surf=surf, NBLOOP=1e6,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35)
m0     = S.run(wl=wl,  NBPHOTONS=NB, atm=pro_clr, sensor=sensors, le=None, surf=surf, NBLOOP=1e6,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35)

In [ ]:
_=smartg_view(m3D.sub({'sensor index':0}))
_=smartg_view(m0.sub({'sensor index':0}))

## Cloud

In [ ]:
# Cloud Phase matrix
cld_phase = read_mlut('/home/did/RTC/SMART-G/smartg/validation/IPRT/watercloud.mie.nc')
ntheta = cld_phase['ntheta'][0,0,0]
wl     = [670.]
l      = []
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() + cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() - cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,2,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,3,:ntheta].ravel())
# Normalization to 2
p      = cld_phase['phase'][:,:,0,:ntheta].ravel()
theta  = cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]
mu     = np.cos(np.radians(theta))
NORM   = np.trapz(p,-mu)
data   = np.array(l) 
print ('phase function norm: ', NORM)
#data = np.array(l) * (2./Norm)
# 2) store data in a LUT object with 4 dimensions, the z dimension is restricted to one level of altitude 0
pha_cld = LUT(data[:, ::-1], names=['stk', 'theta_atm'],
          axes=[None, cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]]
         )

In [ ]:
cloud_LUT=read_mlut('/home/did/RTC/SMART-G/smartg/data/opt_liquid_cloud.nc')

### Step cloud C1

In [ ]:
#alternative reff = 2 micron phase function
wl = 1020.
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl)['phase_atm'].sub()[0,:,:]

In [ ]:
Nx = 2
Ny = 1
Nz = 1
Dx = 0.5/Nx
Dy = 1e5
Dz = 0.25

(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Nz=Nz, Dx=Dx, Dy=Dy, Dz=Dz, periodic=True,
                SAT_ALTITUDE=120, HORIZ_EXTENT_LENGTH=0)

Ncell   = NX*NY*NZ
Natm    = 3 # 2 cloud cells + void
# cells optical properties indices
iopt    = np.array([0, 1, 2, 1], dtype=int32)
# optical properties 
ptc_sca = np.array([2./0.25, 0., 18./0.25])
ptc_ssa = np.array([0.99, 1., 0.99])
ray_sca = np.zeros_like(ptc_sca)
gas_abs = np.zeros_like(ptc_sca)
ipha    = np.zeros_like(ptc_sca, dtype=int32)

In [ ]:
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax

In [ ]:
atm3D_cld = AtmAFGL('afglt',
                    grid        = zgrid,
                    prof_ray    = ray_sca[None,:],
                    prof_abs    = gas_abs[None,:],
                    prof_aer    = (ptc_sca[None,:], ptc_ssa[None,:]),
                    prof_phases = (ipha[None,:], [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )
wl = [800.]

pro3D_cld = atm3D_cld.calc(wl).describe()

In [ ]:
# Sun position
SAA    = 180.
SZA    = 20.
# Sensors position
Npix   = 32
POSZ   = 0.25
THDEG  = 120.
PHDEG  = 135.
# pixels centers on the ground
x0    = np.linspace(xgrid[0]+Dx/2., xgrid[-1]-Dx/2., num=Npix) # central domain (cumulus) boundaries
y0    = np.array([0.])
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ
# cells indices
icells = locate_3D_cells(pro3D_cld, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,ICELL in zip(xx.ravel(), yy.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

In [ ]:
S3D    = Smartg(opt3D=True,  alt_pp=True, alis=False) # 3D

In [ ]:
%%time
#S3D    = Smartg(opt3D=True,  alt_pp=True) # 3D
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}

NB     = len(sensors) * 1e5
surf   = None
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_cld, sensor=sensors, le=le, surf=surf, NBLOOP=1e5,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35, alis_options={'nlow':-1})

In [ ]:
(m3D['I_up (TOA)'].sub()[:,0,0]/np.pi*1e3).plot(vmin=0, fmt='-')

In [ ]:
(m3D['V_up (TOA)'].sub()[:,0,0]/np.pi*1e3).plot( fmt='-')

In [ ]:
len(sensors)

### Cubic cloud C2

In [ ]:
#alternative reff = 2 micron phase function
wl = 1020.
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl)['phase_atm'].sub()[0,:,:]

In [ ]:
#xgrid = np.array([-1e5, -3.5, -0.5, 0.5, 3.5, 1e5])
#ygrid = np.array([-1e5, -3.5, -0.5, 0.5, 3.5, 1e5])
#xgrid = np.array([ -3.5, -0.5, 0.5, 3.5])
#ygrid = np.array([ -3.5, -0.5, 0.5, 3.5])
Nx=1
Ny=1
Dx=1.
Dy=1.
zgrid = np.array([0., 2., 3., 5., 120.])

(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=1e5)

Ncell   = NX*NY*NZ
#Natm    = 2 # 1 cloud cell + void
# cells optical properties indices
cloud_cell = np.ravel_multi_index((1, 1, 1), dims=(NX,NY,NZ), order='C')
iopt    = np.zeros(Ncell, dtype=int32)
iopt[cloud_cell] = 1
# optical properties 
ptc_sca = np.array([0., 10.])
ptc_ssa = np.array([1., 1.])
ray_sca = np.zeros_like(ptc_sca)
gas_abs = np.zeros_like(ptc_sca)
ipha    = np.zeros_like(ptc_sca, dtype=int32)

In [ ]:
atm3D_cld = AtmAFGL('afglt',
                    grid        = np.array([120.,0]),
                    #grid        = zgrid,
                    prof_ray    = ray_sca[None,:],
                    prof_abs    = gas_abs[None,:],
                    prof_aer    = (ptc_sca[None,:], ptc_ssa[None,:]),
                    prof_phases = (ipha[None,:], [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )
wl = [800.]

pro3D_cld = atm3D_cld.calc(wl).describe()

In [ ]:
# Sun position
SAA    = 0.
SZA    = 40.
# Sensors position
Npix   = 70
POSZ   = 5.0001
POSX   = 3.4999
POSY   = 3.4999
THDEG  = 180.
PHDEG  = 0.
# pixels centers on the ground
#x0    = np.linspace(xgrid[1], xgrid[-2], num=Npix) # central domain (cumulus) boundaries
#y0    = np.linspace(ygrid[1], ygrid[-2], num=Npix)
x0    = np.linspace(-POSX, POSX, num=Npix) # central domain (cumulus) boundaries
y0    = np.linspace(-POSY, POSY, num=Npix)
z0    = np.linspace( POSZ,   5 , num=Npix)
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ

#xx,zz = np.meshgrid(x0, z0)
#yy    = np.zeros_like(xx) + POSY
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid, zgrid, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,POSZ,ICELL in zip(xx.ravel(), yy.ravel(), zz.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

In [ ]:
%%time
S3D    = Smartg(opt3D=True,  alt_pp=True, alis=True) # 3D
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}

NB     = len(sensors) * 1e4
surf   = LambSurface(ALB=0.2)
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_cld, sensor=sensors, le=le, surf=surf, NBLOOP=1e5,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35, alis_options={'nlow':-1})

In [ ]:
print(NB)
imshow(m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:], cmap='jet')
colorbar()
figure()
imshow(m3D['Q_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:], cmap='jet')
colorbar()

### Cumulus C3

In [ ]:
#alternative reff = 2 micron phase function
wl = 670.
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl)['phase_atm'].sub()[0,:,:]

# Cloud Phase matrix
# alternative reff = 5 micron phase function
cloud_sub = cloud_LUT.sub({'effective_radius':Idx(5.),'effective_variance':0, 'wavelength':Idx(0.670)})
pha = np.stack([(cloud_sub['p11_phase_function']+cloud_sub['p21_phase_function'])[::-1],
               (cloud_sub['p11_phase_function']-cloud_sub['p21_phase_function'])[::-1],
                cloud_sub['p34_phase_function'][::-1], cloud_sub['p44_phase_function'][::-1]])
pha_cld = LUT(pha, axes=[None, cloud_sub.axis('phase_angles')[::-1]], names=['stk', 'theta_atm'])

In [ ]:
# C3 IPRT Cumulus data
cumulus_f='/home/did/RTC/SMART-G/smartg/validation/IPRT/cumulus.dat'
f       = open(cumulus_f, 'r')
f.readline(); 
l       = f.readline().split()
Nx, Ny, Nz, _ = list(map(int, l)) # number of cumulus cells in X, Y, X
Nl      = Nz + 1                  # number of levels in Z
l       = f.readline()
g_info  = list(map(float, l.split()))
Dx, Dy  = g_info[0:2]              # horizontal grid interval in km
#zgrid   = np.array(g_info[2:][::-1])# vertical grid in km
zgrid   = np.array(g_info[2:])# vertical grid in km
assert Nl==zgrid.size

zgrid = np.concatenate([zgrid, np.array([120.])])
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Dx=Dx, Dy=Dy, z=zgrid, SAT_ALTITUDE=120, periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ

In [ ]:
(idx,idy,idz), (NX,NY,NZ), neigh, pmin, pmax

In [ ]:
# cumulus geometrical and optical data
cumulus = np.loadtxt(cumulus_f, skiprows=3, dtype=np.float32)
ptc_sca = cumulus[:,3]            # km-1
# we add a zero optical property for void
ptc_sca = np.concatenate([np.array([0.]), ptc_sca])
ptc_ssa = np.ones_like( ptc_sca)
ray_sca = np.zeros_like(ptc_sca)  # No Rayleigh
gas_abs = np.zeros_like(ptc_sca)  # No absorption
ipha    = np.zeros_like(ptc_sca, dtype=int32)

# indexing of C3 cells into SMART-G grid
cum_geo = cumulus[:,:3].astype(int32)-1
cum_cell= np.ravel_multi_index((cum_geo[:,0], cum_geo[:,1], cum_geo[:,2]),
                    dims=(NX, NY, NZ))
# cells optical properties
iopt    = np.zeros(Ncell, dtype=int32)
iabs    = np.zeros(Ncell, dtype=int32)
iopt[cum_cell] = np.arange(len(cum_cell)) + 1 # number zero is reserved for void
Nopt    = iopt.max()+1
atm_arr = np.zeros((Nopt,9))
atm_arr[:,0] = np.arange(Nopt)[::-1]
np.savetxt('./tmp.dat', atm_arr)

In [ ]:
atm3D_cld = AtmAFGL('./tmp.dat', US=False,
                    grid        = np.arange(Nopt),
                    prof_ray    = ray_sca[None,:],
                    #prof_abs    = gas_abs[None,:],
                    prof_aer    = (ptc_sca[None,:], ptc_ssa[None,:]),
                    prof_phases = (ipha[None,:], [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )

pro3D_cld = atm3D_cld.calc(wl).describe()

In [ ]:
ray_sca.shape, ptc_sca[iopt[cum_cell]]

In [ ]:
# Sun position
SAA    = 180.
SZA    = 40.
# Sensors position
Npix   = 100
POSZ   = 30
THDEG  = 180.
PHDEG  = 0.
# pixels centers on the ground
x0     = np.linspace(-6.67/2., 6.67/2, num=Npix) # central domain (cumulus) boundaries
y0     = np.linspace(-6.67/2., 6.67/2, num=Npix)
#
xx,yy  = np.meshgrid(x0, y0)
zz     = np.zeros_like(xx) + POSZ

In [ ]:
%%time
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid,zgrid, xx.ravel(), yy.ravel(), zz.ravel())
#icells_old = locate_3D_cells(pro3D_cld, xx.ravel(), yy.ravel(), zz.ravel())
sensors=[]
for POSX,POSY,ICELL in zip(xx.ravel(), yy.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

S3D    = Smartg(opt3D=True,  alt_pp=True, alis=False, back=True) # 3D
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
#ALB    = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.grass.green.solid.gras.spectrum.txt')
#env    = Environment(ENV=1, ALB=ALB, ENV_SIZE=2, X0=-1.5)
env    = None

In [ ]:
%%time
NB     = len(sensors) * 1e4
#surf   = RoughSurface(WIND=5)
surf   = LambSurface(ALB=0.2)
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_cld, sensor=sensors, le=le, surf=surf, NBLOOP=1e6, env=env,
               stdev=False, NF=1e4)

In [ ]:
f=figure(figsize=(8,8))
print(NB)
imshow(m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:],vmax=1, cmap=cm.Blues_r)
cbar = colorbar(shrink=0.8, orientation='vertical')
cbar.ax.set_xlabel(mdesc('I_up (TOA)'))
f.tight_layout()
f.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C3_case5_image.png', dpi=200)
figure(figsize=(6,6))
I=m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
Q=m3D['Q_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
U=m3D['U_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
V=m3D['V_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
imshow(np.sqrt(Q*Q+U*U+V*V)/I*100, vmax=5,cmap=cm.RdBu_r)
colorbar()

## ALIS

### Spectral range

In [ ]:
#Solar Spectrum raw data
datas = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_full.dat')
#datas = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_0.1nm.dat')

In [ ]:
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(685., 698., datas, dl=0.001, dls=10.)
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(405., 490., datas, dl=0.01, dls=10.)
wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(1990., 2100., datas, dl=0.01, dls=10.)
# Solar spectrum
Es = Es_LUT[Idx(wl)]
Es_RRS = Es_LUT[Idx(wl_RRS, fill_value='extrema')]
# Ring nromalized spectrum (Wagner et al., 2009, AMT)
_, LRRS = L2d_inv(wl, 90., 243.)
f_RRS_norm  = np.sum(Es_RRS * LRRS , axis=1) / Es - 1.

### Simple atmosphere

In [ ]:
# vertical grid in descending order as usual for SmartG 1D
znew = np.linspace(50., 0. , num=26)
#znew = np.array([100., 99., 20., 5., 3., 2., 0.])
Nx=1
Ny=1
Dx=1
Dy=1
# vertical grid in ascending order for 3D grid
zgrid = znew[::-1]

# grid building
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ

idx_cloud  = [1]
idy_cloud  = [1]
idz_cloud  = [1]
cloud_cell = np.ravel_multi_index((idx_cloud, idy_cloud, idz_cloud), 
                                  dims=(NX,NY,NZ), order='C')

In [ ]:
# vertical grid in descending order as usual for SmartG 1D
znew = np.linspace(50., 0. , num=26)
#znew = np.array([100., 99., 20., 5., 3., 2., 0.])
Nx=1
Ny=1
Dx=1e5
Dy=1e5
# vertical grid in ascending order for 3D grid
zgrid = znew[::-1]

# grid building
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=0)
Ncell   = NX*NY*NZ

# cells optical properties
iopt           = np.zeros(Ncell, dtype= int32)
#Rayleigh
sca_ray        = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(wls), 'OD_r')
## Mapping optical propoerties and cells
# 1 clear profile + number of cloud cells opt. prop. (here 1 per cell)
Nopt       = NZ + 1
iopt       = np.zeros(Ncell, dtype=int32)
iabs       = np.zeros_like(iopt)
iopt[:]    = np.arange(Nopt)[NZ-idz] # Scattering depending on Z for clear atmosphere (Rayleigh)
iabs[:]    = np.arange(Nopt)[NZ-idz] # Absorption depending on Z only

atm3D_clr = AtmAFGL('afglt', O3=0., NO2=False,
                    grid        = np.arange(Nopt),
                    prof_ray    = sca_ray,
                    cells       = (iopt, iabs, pmin, pmax, neigh)
                   )

pro3D_clr = atm3D_clr.calc(wls)
# absorption (only gas)
ao2_int = get_o2_abs(znew, wl)

In [ ]:
SAA    = 180.
SZA    = 40.
# Sensors position
Npix   = 70
POSZ   = 49.0001
#POSZ   = 0.
POSX   = 0.
POSY   = 0.
#THDEG  = 20.
THDEG  = 180.
PHDEG  = 140.
# cells indices
x      = np.array([POSX])
y      = np.array([POSY])
z      = np.array([POSZ])
icells = locate_3Dregular_cells(xgrid, ygrid, zgrid, x , y, z)
sensors = Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=icells)

In [ ]:
%%time
# Spectra
zoom          = (685.75, 688.00)
sigma         = ao2_int[:,1:]
sigma[:]=0
NWS           = wls.size
#sigma         = gas_abs[:,1:]
S3Da          = Smartg(opt3D=True,  alt_pp=True, alis=True) # 3D
le            = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
ALB           = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.trees.deciduous.solid.decidou.spectrum.txt')
fig,ax        = subplots(nrows=5, ncols=1) 
fig.set_size_inches(24,15)
surf          = LambSurface(ALB=1.)
surfref       = LambSurface(ALB=ALB.get(wls[0]))
NL            = Nopt-1 # number of opt properties
NB            = 1e4
for i,sensor in enumerate([sensor]):
    m       = S3Da.run(wl=wls,  NBPHOTONS=NB, atm=pro3D_clr, sensor=sensor, le=le, surf=surf, NBLOOP=1e4,
               stdev=False, NF=1e4, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0})
    #mref    = S3D.run(wl=wls[0],  NBPHOTONS=NB*100, atm=pro3D0, sensor=sensor, le=le, surf=surfref, NBLOOP=1e5,
    #           stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

    w     = m['disth_up (TOA)'][:, NL+4:-3]
    ngood = np.sum(w[:,0]!=0)

    S       = np.zeros((ngood,4),dtype=np.float32) 
    cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
    S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
    w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
    nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
    nref    = m['disth_up (TOA)'][:ngood,      -2  ]
    nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

    XBLOCK  = 512
    XGRID   = 512
    NT      = XBLOCK*XGRID       # Maximum Number of threads
    NPHOTON = cd.shape[0]        # Number of photons

    NLAYER  = sigma.shape[1]     # Number of vertical layer
    NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
    NGROUP  = NT//NWVL           # Number of groups of photons
    NTHREAD = NGROUP*NWVL        # Number of threads used
    NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
    NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
    NWVL_LOW = NWS

    print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
    sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
    sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

    alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32) 

    cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    #cd_in        = cd[:,:NLAYER].reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
    weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
    nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
    nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
    nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

    res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
    reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                          np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                          to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                          to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                          block=(XBLOCK,1,1),grid=(XGRID,1,1))
    I1      = res_out.get()/m[0][0,0]
    #norm1   = I1[0,0]/mref['I_up (TOA)'].data
    P_Raman = res_rrs.get()[0,:]/res_out.get()[0,:]
    #print(norm1)
    norm1 = 1
    I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))
    Q1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('Q_up (TOA)'))
    U1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('U_up (TOA)'))
    DoLP_LUT = ((Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)/I1_LUT)*100
    DoLP_LUT.desc='DoLP (%)'
    P_Raman_LUT = LUT(P_Raman, axes=[wl], names=['wavelength'], desc=r'$P_{Raman}$')
    Rel     = Es * I1_LUT[Idx(wl,fill_value='extrema')]
    Rel_llp = Es_RRS * I1_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Rel_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rel_out = Rel * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    R_tot   = R_RRS + Rel - Rel_out
    
    sca(ax[0,0])
    I1_LUT.plot(label='%d'%i)
    ylim(0,0.4)
    sca(ax[0,1])
    I1_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,0.4)
    
    sca(ax[1,0])
    ff = R_tot
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[1,1])
    ff = R_tot
    plot(wl, ff ,'-')
    xlim(zoom)
    xlabel('wavelength (nm)')
    grid(True)
    
    sca(ax[2,0])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, RRS in)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[2,1])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    xlim(zoom)
    grid(True)

    sca(ax[3,0])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylabel('$Ring spectrum (Tot/Elastic)$')
    ylim(0.95,1.05)
    grid(True)
    sca(ax[3,1])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylim(0.95,1.05)
    grid(True)
    xlim(zoom)
    
    sca(ax[4,0])
    DoLP_LUT.plot(label='%d'%i)
    ylim(0,40)
    sca(ax[4,1])
    DoLP_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,40)
legend()

In [ ]:
ao2_int[-1,0:]
NL,NLAYER,Nopt,w.shape,NWS,NWVL_LOW
m['disth_up (TOA)'][0,NL+4:-3]

### C2 cloud

#### 3D Grid

In [ ]:
# vertical grid in descending order as usual for SmartG 1D
znew = np.linspace(50., 0. , num=26)
#znew = np.array([100., 99., 20., 5., 3., 2., 0.])
Nx=1
Ny=1
Dx=1
Dy=1
# vertical grid in ascending order for 3D grid
zgrid = znew[::-1]

# grid building
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ

# cloud cells
idx_cloud  = [1,1,1,1]
idy_cloud  = [1,1,1,1]
idz_cloud  = [0,1,2,3]
'''idx_cloud  = [1]
idy_cloud  = [1]
idz_cloud  = [1]'''
cloud_cell = np.ravel_multi_index((idx_cloud, idy_cloud, idz_cloud), 
                                  dims=(NX,NY,NZ), order='C')

#### 1D opt.  prop.

In [ ]:
# absorption (only gas)
ao2_int = get_o2_abs(znew, wl)

In [ ]:
cloud_LUT=read_mlut('/home/did/RTC/SMART-G/smartg/data/opt_liquid_cloud.nc')

In [ ]:
## scattering optical properties
wl_ref  = (wls[0]+wls[-1])/2.
# cloud
reff = 5.  # mic.
kext = 5.  # km-1
#kext = 0.  # km-1
cloud_sub   = cloud_LUT.sub({'effective_radius':Idx(reff),'effective_variance':0, 'wavelength':Idx(wl_ref*1e-3)})
pha = np.stack([(cloud_sub['p11_phase_function']+cloud_sub['p21_phase_function'])[::-1],
                (cloud_sub['p11_phase_function']-cloud_sub['p21_phase_function'])[::-1],
                 cloud_sub['p34_phase_function'][::-1],
                 cloud_sub['p44_phase_function'][::-1]])
pha_cld     = LUT(pha, axes=[None, cloud_sub.axis('phase_angles')[::-1]], names=['stk', 'theta_atm'])

cloud_sub   = cloud_LUT.sub({'effective_radius':Idx(reff),'effective_variance':0})
sca_cld     = cloud_sub['normed_ext_coeff' ][Idx(wls, fill_value='extrema')]/\
              cloud_sub['normed_ext_coeff' ][Idx(wl_ref, fill_value='extrema')]*kext
#Rayleigh
sca_ray     = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(wls), 'OD_r')

# Concatenation
sca_cld3D = np.concatenate([np.zeros_like(sca_ray), np.stack([sca_cld]*cloud_cell.size, axis=1)], axis=1)
sca_ray3D = np.concatenate([sca_ray, sca_ray[:,NZ-idz[cloud_cell]]], axis=1)
ssa_cld3D = np.ones_like(sca_cld3D)
ipha3D    = np.zeros_like(sca_cld3D, dtype=int32)
#ipha3D[:, iopt[cloud_cell]] = 1

In [ ]:
## Mapping optical propoerties and cells
# 1 clear profile + number of cloud cells opt. prop. (here 1 per cell)
Nopt       = NZ + 1 + cloud_cell.size
iopt       = np.zeros(Ncell, dtype=int32)
iabs       = np.zeros_like(iopt)
iopt[:]    = np.arange(Nopt)[NZ-idz] # Scattering depending on Z for clear atmosphere (Rayleigh)
iabs[:]    = np.arange(Nopt)[NZ-idz] # Absorption depending on Z only
iopt[cloud_cell] = NZ + 1 + np.arange(cloud_cell.size)  # Scattering depending on cloud cell

In [ ]:
atm3D = AtmAFGL('afglt',
                    grid        = np.arange(Nopt),
                    prof_ray    = sca_ray3D,
                    prof_aer    = (sca_cld3D, ssa_cld3D),
                    prof_phases = (ipha3D, [pha_cld]),
                    cells       = (iopt, iabs, pmin, pmax, neigh)
                   )
pro3D  = atm3D.calc(wls)

# Sun position
SAA    = 180.
SZA    = 40.
# Sensors position
Npix   = 70
POSZ   = 49.0001
#POSZ   = 0.
POSX   = 3.4999
POSY   = 3.4999
#THDEG  = 20.
THDEG  = 180.
PHDEG  = 140.
# pixels centers on the ground
x0    = np.linspace(-POSX, POSX, num=Npix) # central domain (cumulus) boundaries
y0    = np.linspace(-POSY, POSY, num=Npix)
z0    = np.linspace( POSZ,   5 , num=Npix)
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid, zgrid, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,POSZ,ICELL in zip(xx.ravel(), yy.ravel(), zz.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))
    
sensor_cld1 = Sensor(POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, 0., 0., POSZ))
sensor_cld2 = Sensor(POSX=-0.4, POSY=-0.4, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, -0.4, -0.4, POSZ))

sensor_grd = Sensor(POSX = -POSX, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, -POSX, 0., POSZ))

In [ ]:
%%time
# Spectra
zoom          = (685.75, 688.00)
sigma         = ao2_int[:,:]
NWS           = wls.size
#sigma         = gas_abs[:,1:]
S3Da          = Smartg(opt3D=True,  alt_pp=True, alis=True) # 3D
le            = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
ALB           = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.trees.deciduous.solid.decidou.spectrum.txt')
fig,ax        = subplots(nrows=5, ncols=2) 
fig.set_size_inches(24,15)
surf          = LambSurface(ALB=1.)
surfref       = LambSurface(ALB=ALB.get(wls[0]))
NL            = Nopt-1 # number of opt properties
NB            = 1e5
for i,sensor in enumerate([sensor_cld1, sensor_grd]):
    m       = S3Da.run(wl=wls,  NBPHOTONS=NB, atm=pro3D, sensor=sensor, le=le, surf=surf, NBLOOP=1e4,
               stdev=False, NF=1e4, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0})
    #mref    = S3D.run(wl=wls[0],  NBPHOTONS=NB*100, atm=pro3D0, sensor=sensor, le=le, surf=surfref, NBLOOP=1e5,
    #           stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

    w     = m['disth_up (TOA)'][:, NL+4:-3]
    ngood = np.sum(w[:,0]!=0)

    S       = np.zeros((ngood,4),dtype=np.float32) 
    cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
    S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
    w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
    nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
    nref    = m['disth_up (TOA)'][:ngood,      -2  ]
    nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

    XBLOCK  = 512
    XGRID   = 512
    NT      = XBLOCK*XGRID       # Maximum Number of threads
    NPHOTON = cd.shape[0]        # Number of photons

    NLAYER  = sigma.shape[1]     # Number of vertical layer
    NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
    NGROUP  = NT//NWVL           # Number of groups of photons
    NTHREAD = NGROUP*NWVL        # Number of threads used
    NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
    NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
    NWVL_LOW = NWS

    print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
    sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
    sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

    alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32) 

    #cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    cd_in        = cd[:,:NLAYER].reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
    weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
    nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
    nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
    nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

    res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
    reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                          np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                          to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                          to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                          block=(XBLOCK,1,1),grid=(XGRID,1,1))
    I1      = res_out.get()/m[0][0,0]
    #norm1   = I1[0,0]/mref['I_up (TOA)'].data
    P_Raman = res_rrs.get()[0,:]/res_out.get()[0,:]
    #print(norm1)
    norm1 = 1
    I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))
    Q1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('Q_up (TOA)'))
    U1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('U_up (TOA)'))
    DoLP_LUT = ((Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)/I1_LUT)*100
    DoLP_LUT.desc='DoLP (%)'
    P_Raman_LUT = LUT(P_Raman, axes=[wl], names=['wavelength'], desc=r'$P_{Raman}$')
    Rel     = Es * I1_LUT[Idx(wl,fill_value='extrema')]
    Rel_llp = Es_RRS * I1_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Rel_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rel_out = Rel * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    R_tot   = R_RRS + Rel - Rel_out
    
    sca(ax[0,0])
    I1_LUT.plot(label='%d'%i)
    ylim(0,0.4)
    sca(ax[0,1])
    I1_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,0.4)
    
    sca(ax[1,0])
    ff = R_tot
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[1,1])
    ff = R_tot
    plot(wl, ff ,'-')
    xlim(zoom)
    xlabel('wavelength (nm)')
    grid(True)
    
    sca(ax[2,0])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, RRS in)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[2,1])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    xlim(zoom)
    grid(True)

    sca(ax[3,0])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylabel('$Ring spectrum (Tot/Elastic)$')
    ylim(0.95,1.05)
    grid(True)
    sca(ax[3,1])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylim(0.95,1.05)
    grid(True)
    xlim(zoom)
    
    sca(ax[4,0])
    DoLP_LUT.plot(label='%d'%i)
    ylim(0,40)
    sca(ax[4,1])
    DoLP_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,40)
legend()

In [ ]:
# vertical grid in descending order as usual for SmartG 1D
znew = np.linspace(50., 0. , num=26)
#znew = np.array([100., 99., 20., 5., 3., 2., 0.])
Nx=1
Ny=1
Dx=1.
Dy=1.
# vertical grid in ascending order for 3D grid
zgrid = znew[::-1]

# grid building
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ

# cloud cells
idx_cloud = 1
idy_cloud = 1
idz_cloud = 1

Nopt      = len(zgrid) + 1 # one clear atmosphere + 1 cloud cell 


cloud_cell = np.ravel_multi_index((idx_cloud, idy_cloud, idz_cloud), dims=(NX,NY,NZ), order='C')
iopt       = np.zeros(Ncell, dtype=int32)
iopt[:]    = np.arange(Nopt)[::-1][idz+1]
iopt[cloud_cell] = Nopt - 1

## optical properties
wl_ref  = 690.
# aerosols
aer     = AeroOPAC('urban', 0., wl_ref)
pha_aer = AtmAFGL('afglt', comp=[aer]).calc(wl_ref, NBTHETA=10000)['phase_atm'].sub()[0,:,:]
cld     = CloudOPAC('wc.sol', 3., znew[-2], znew[-3], 10., wl_ref)
#pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl_ref, NBTHETA=10000)['phase_atm'].sub()[0,:,:]

atm1D_clr  = AtmAFGL('afglt', O3=0., comp=[aer], NO2=True, grid=znew).calc(wls)
atm1D_cld  = AtmAFGL('afglt', O3=0., NO2=True, comp=[cld], grid=znew).calc(wls)
atm1D_clrA  = AtmAFGL('afglt', O3=0., comp=[aer], NO2=True, prof_abs=ao2_int, grid=znew).calc(wl)
atm1D_cldA  = AtmAFGL('afglt', O3=0., NO2=True, comp=[cld], prof_abs=ao2_int, grid=znew).calc(wl)
#atm1D_clrA  = AtmAFGL('afglt', O3=0., comp=[aer], NO2=True, grid=znew).calc(wl)
#atm1D_cldA  = AtmAFGL('afglt', O3=0., NO2=True, comp=[cld], grid=znew).calc(wl)
ptc_sca    = np.concatenate([od2k(atm1D_clr, 'OD_p')[:,:], od2k(atm1D_cld, 'OD_p') [:,-idz_cloud-1:-idz_cloud]], axis=1)
ptc_ssa    = np.concatenate([atm1D_clr['ssa_p_atm'] [:,:], atm1D_cld['ssa_p_atm']  [:,-idz_cloud-1:-idz_cloud]], axis=1)
ray_sca    = np.concatenate([od2k(atm1D_clr, 'OD_r')[:,:], od2k(atm1D_cld, 'OD_r') [:,-idz_cloud-1:-idz_cloud]], axis=1)
gas_abs    = np.concatenate([od2k(atm1D_clrA, 'OD_g')[:,:], od2k(atm1D_cldA, 'OD_g') [:,-idz_cloud-1:-idz_cloud]], axis=1)

atm1D_clr0  = AtmAFGL('afglt', O3=0., comp=[aer], NO2=False, grid=znew).calc(wls[0])
atm1D_cld0  = AtmAFGL('afglt', O3=0., NO2=False, comp=[cld], grid=znew).calc(wls[0])
ptc_sca0    = np.concatenate([od2k(atm1D_clr0, 'OD_p')[:,:], od2k(atm1D_cld0, 'OD_p') [:,-idz_cloud-1:-idz_cloud]], axis=1)
ptc_ssa0    = np.concatenate([atm1D_clr0['ssa_p_atm'] [:,:], atm1D_cld0['ssa_p_atm']  [:,-idz_cloud-1:-idz_cloud]], axis=1)
ray_sca0    = np.concatenate([od2k(atm1D_clr0, 'OD_r')[:,:], od2k(atm1D_cld0, 'OD_r') [:,-idz_cloud-1:-idz_cloud]], axis=1)
gas_abs0    = np.concatenate([od2k(atm1D_clr0, 'OD_g')[:,:], od2k(atm1D_cld0, 'OD_g') [:,-idz_cloud-1:-idz_cloud]], axis=1)

ipha0       = np.zeros_like(ptc_sca0, dtype=int32)
ipha0[:, iopt[cloud_cell]] = 1
ipha        = np.zeros_like(ptc_sca, dtype=int32)
ipha[:, iopt[cloud_cell]] = 1

NWS=wls.size
# parameters for 1D linear interpolation of w
f  =  interp1d(wls,np.linspace(0, NWS-1, num=NWS))
iw =  f(wl)
#iw =  f(wl_RRS.ravel())
iwls_in = np.floor(iw).astype(int8)        # index of lower wls value in the wls array, 
wwls_in = (iw-iwls_in).astype(np.float32)  # floating proportion between iwls and iwls+1
# special case for NWS
ii = np.where(iwls_in==(NWS-1))
iwls_in[ii] = NWS-2
wwls_in[ii] = 1.

In [ ]:
# Cloud Phase matrix
# alternative reff = 5 micron phase function
cloud_sub = cloud_LUT.sub({'effective_radius':Idx(5.),'effective_variance':0, 'wavelength':Idx(0.670)})
pha = np.stack([(cloud_sub['p11_phase_function']+cloud_sub['p21_phase_function'])[::-1],
               (cloud_sub['p11_phase_function']-cloud_sub['p21_phase_function'])[::-1],
                cloud_sub['p34_phase_function'][::-1], cloud_sub['p44_phase_function'][::-1]])
pha_cld = LUT(pha, axes=[None, cloud_sub.axis('phase_angles')[::-1]], names=['stk', 'theta_atm'])

In [ ]:
ray_sca[0,:], ptc_sca[0,:], gas_abs[0,:], atm1D_cld.axis('z_atm'), idz, idz_cloud, iopt
iopt, pmin[2,cloud_cell], iopt[cloud_cell], ptc_sca[0,iopt[cloud_cell]], gas_abs[0,iopt[cloud_cell]],  ray_sca[0,iopt[cloud_cell]], Nopt
ptc_ssa[0,iopt[cloud_cell]]

In [ ]:
atm3D = AtmAFGL('afglt',
                    grid        = np.arange(Nopt),
                    prof_ray    = ray_sca,
                    prof_aer    = (ptc_sca, ptc_ssa),
                    prof_phases = (ipha, [pha_aer,pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )

pro3D  = atm3D.calc(wls)
atm3D0 = AtmAFGL('afglt',
                    grid        = np.arange(Nopt),
                    prof_ray    = ray_sca0,
                    prof_abs    = gas_abs0,
                    prof_aer    = (ptc_sca0, ptc_ssa0),
                    prof_phases = (ipha0, [pha_aer,pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )

pro3D0 = atm3D0.calc(wls[0])
# Sun position
SAA    = 180.
SZA    = 40.
# Sensors position
Npix   = 70
POSZ   = 49.0001
#POSZ   = 0.
POSX   = 3.4999
POSY   = 3.4999
#THDEG  = 20.
THDEG  = 180.
PHDEG  = 140.
# pixels centers on the ground
x0    = np.linspace(-POSX, POSX, num=Npix) # central domain (cumulus) boundaries
y0    = np.linspace(-POSY, POSY, num=Npix)
z0    = np.linspace( POSZ,   5 , num=Npix)
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid, zgrid, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,POSZ,ICELL in zip(xx.ravel(), yy.ravel(), zz.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))
    
sensor_cld1 = Sensor(POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, 0., 0., POSZ))
sensor_cld2 = Sensor(POSX=-0.4, POSY=-0.4, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, -0.4, -0.4, POSZ))

sensor_grd = Sensor(POSX = -POSX, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, -POSX, 0., POSZ))

In [ ]:
#Monochromatic image
le      = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
S3D     = Smartg(opt3D=True,  alt_pp=True, alis=False) # 3D
ALB     = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.trees.deciduous.solid.decidou.spectrum.txt')
surf    = LambSurface(ALB=ALB.get(wls[0]))
NB      = len(sensors) * 1e3
m0      = S3D .run(wl=wls[0],  NBPHOTONS=NB, atm=pro3D.sub({'wavelength':0}), sensor=sensors, le=le, surf=surf, NBLOOP=1e5,
               stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

In [ ]:
#Monochromatic image NO2
le      = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
S3D     = Smartg(opt3D=True,  alt_pp=True, alis=False) # 3D
ALB     = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.trees.deciduous.solid.decidou.spectrum.txt')
surf    = LambSurface(ALB=ALB.get(wls[0]))
NB      = len(sensors) * 1e5
m0      = S3D .run(wl=wls[0],  NBPHOTONS=NB, atm=pro3D0, sensor=sensors, le=le, surf=surf, NBLOOP=1e6,
               stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

In [ ]:
# O2
f=figure(figsize=(6,6))
im = m0['I_up (TOA)'].data.reshape(Npix,Npix,1)[::-1,:,-1]
imshow(im, vmin=0 , vmax=0.39, cmap=cm.Blues_r)
cbar = colorbar(shrink=0.8, orientation='vertical')
cbar.ax.set_xlabel(mdesc('I_up (TOA)'))
f.tight_layout()
#f.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C2_case5_O2_image.png', dpi=200)

In [ ]:
# NO2
f=figure(figsize=(6,6))
im = m0['I_up (TOA)'].data.reshape(Npix,Npix,1)[::-1,:,-1]
imshow(im, vmin=0 , cmap=cm.Blues_r)
cbar = colorbar(shrink=0.8, orientation='vertical')
cbar.ax.set_xlabel(mdesc('I_up (TOA)'))
f.tight_layout()
f.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C2_case5_NO2_image.png', dpi=200)

In [ ]:
f=figure(figsize=(6,4))
phases = [pha_cld, pha_aer]
ip  = np.array([p.axis('theta_atm').size for p in phases]).argmax()
theta = phases[ip].axis('theta_atm')
pha = np.stack([p[:,Idx(theta)] for p in phases])
pha.shape, pha_cld.data.shape, pha_aer.data.shape
semilogy(theta, pha[1,0,:],label='urban aerosol')
semilogy(theta, pha[0,0,:],label=r'liquid cloud $r_{eff}=5\mu m$')
grid()
xlim(0,180)
ylim(1e-2,1e4)
xlabel(r'$\theta$')
ylabel(r'$P(\theta)$')
legend()
f.tight_layout()
f.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C2_case5_phase_function.png', dpi=200)

In [ ]:
%%time
# Spectra
zoom          = (685.75, 688.00)
sigma         = ao2_int[:,:]
NWS           = wls.size
#sigma         = gas_abs[:,1:]
S3Da          = Smartg(opt3D=True,  alt_pp=True, alis=True) # 3D
fig,ax        = subplots(nrows=5, ncols=2) 
fig.set_size_inches(24,15)
surf          = LambSurface(ALB=1.)
surfref       = LambSurface(ALB=ALB.get(wls[0]))
NL            = Nopt-1 # number of opt properties
NB            = 1e3
for i,sensor in enumerate([sensor_cld1, sensor_grd]):
    m       = S3Da.run(wl=wls,  NBPHOTONS=NB, atm=pro3D, sensor=sensor, le=le, surf=surf, NBLOOP=1e4,
               stdev=False, NF=1e4, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0})
    #mref    = S3D.run(wl=wls[0],  NBPHOTONS=NB*100, atm=pro3D0, sensor=sensor, le=le, surf=surfref, NBLOOP=1e5,
    #           stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

    w     = m['disth_up (TOA)'][:, NL+4:-3]
    ngood = np.sum(w[:,0]!=0)

    S       = np.zeros((ngood,4),dtype=np.float32) 
    cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
    S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
    w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
    nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
    nref    = m['disth_up (TOA)'][:ngood,      -2  ]
    nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

    XBLOCK  = 512
    XGRID   = 512
    NT      = XBLOCK*XGRID       # Maximum Number of threads
    NPHOTON = cd.shape[0]        # Number of photons

    NLAYER  = sigma.shape[1]     # Number of vertical layer
    NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
    NGROUP  = NT//NWVL           # Number of groups of photons
    NTHREAD = NGROUP*NWVL        # Number of threads used
    NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
    NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
    NWVL_LOW = NWS

    print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
    sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
    sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

    alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32) 

    cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
    weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
    nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
    nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
    nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

    res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
    reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                          np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                          to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                          to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                          block=(XBLOCK,1,1),grid=(XGRID,1,1))
    I1      = res_out.get()/m[0][0,0]
    #norm1   = I1[0,0]/mref['I_up (TOA)'].data
    P_Raman = res_rrs.get()[0,:]/res_out.get()[0,:]
    #print(norm1)
    norm1 = 1
    I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))
    Q1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('Q_up (TOA)'))
    U1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('U_up (TOA)'))
    DoLP_LUT = ((Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)/I1_LUT)*100
    DoLP_LUT.desc='DoLP (%)'
    P_Raman_LUT = LUT(P_Raman, axes=[wl], names=['wavelength'], desc=r'$P_{Raman}$')
    Rel     = Es * I1_LUT[Idx(wl,fill_value='extrema')]
    Rel_llp = Es_RRS * I1_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Rel_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rel_out = Rel * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    R_tot   = R_RRS + Rel - Rel_out
    
    sca(ax[0,0])
    I1_LUT.plot(label='%d'%i)
    ylim(0,0.4)
    sca(ax[0,1])
    I1_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,0.4)
    
    sca(ax[1,0])
    ff = R_tot
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[1,1])
    ff = R_tot
    plot(wl, ff ,'-')
    xlim(zoom)
    xlabel('wavelength (nm)')
    grid(True)
    
    sca(ax[2,0])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, RRS in)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[2,1])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    xlim(zoom)
    grid(True)

    sca(ax[3,0])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylabel('$Ring spectrum (Tot/Elastic)$')
    ylim(0.95,1.05)
    grid(True)
    sca(ax[3,1])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylim(0.95,1.05)
    grid(True)
    xlim(zoom)
    
    sca(ax[4,0])
    DoLP_LUT.plot(label='%d'%i)
    ylim(0,40)
    sca(ax[4,1])
    DoLP_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,40)
legend()

In [ ]:
#fig.tight_layout()
fig.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C2_case5_O2_spectre.png', dpi=200)

In [ ]:
%%time
# Spectra NO2
#zoom          = (685.75, 688.00)
zoom          = (430, 431)
sigma         = gas_abs[:,1:]
S3Da          = Smartg(opt3D=True,  alt_pp=True, alis=True) # 3D
fig,ax        = subplots(nrows=5, ncols=2) 
fig.set_size_inches(24,15)
surf          = LambSurface(ALB=1.)
surfref       = LambSurface(ALB=ALB.get(wls[0]))
NL            = Nopt-1 # number of opt properties
NB            = 1e4
for i,sensor in enumerate([sensor_cld1, sensor_grd]):
    m       = S3Da.run(wl=wls,  NBPHOTONS=NB, atm=pro3D, sensor=sensor, le=le, surf=surf, NBLOOP=1e4,
               stdev=False, NF=1e4, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0})
    mref    = S3D.run(wl=wls[0],  NBPHOTONS=NB*100, atm=pro3D0, sensor=sensor, le=le, surf=surfref, NBLOOP=1e5,
               stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

    w     = m['disth_up (TOA)'][:, NL+4:-3]
    ngood = np.sum(w[:,0]!=0)

    S       = np.zeros((ngood,4),dtype=np.float32) 
    cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
    S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
    w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
    nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
    nref    = m['disth_up (TOA)'][:ngood,      -2  ]
    nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

    XBLOCK  = 512
    XGRID   = 512
    NT      = XBLOCK*XGRID       # Maximum Number of threads
    NPHOTON = cd.shape[0]        # Number of photons

    NLAYER  = sigma.shape[1]     # Number of vertical layer
    NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
    NGROUP  = NT//NWVL           # Number of groups of photons
    NTHREAD = NGROUP*NWVL        # Number of threads used
    NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
    NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
    NWVL_LOW = NWS

    print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
    sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
    sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

    alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32) 

    cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
    weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
    nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
    nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
    nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

    res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
    reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                          np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                          to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                          to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                          block=(XBLOCK,1,1),grid=(XGRID,1,1))
    I1      = res_out.get()/m[0][0,0]
    norm1   = I1[0,0]/mref['I_up (TOA)'].data
    P_Raman = res_rrs.get()[0,:]/res_out.get()[0,:]
    print(norm1)
    #norm1 = 1
    I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))
    Q1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('Q_up (TOA)'))
    U1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('U_up (TOA)'))
    DoLP_LUT = ((Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)/I1_LUT)*100
    DoLP_LUT.desc='DoLP (%)'
    P_Raman_LUT = LUT(P_Raman, axes=[wl], names=['wavelength'], desc=r'$P_{Raman}$')
    Rel     = Es * I1_LUT[Idx(wl,fill_value='extrema')]
    Rel_llp = Es_RRS * I1_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Rel_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rel_out = Rel * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    R_tot   = R_RRS + Rel - Rel_out
    
    sca(ax[0,0])
    I1_LUT.plot(label='%d'%i)
    ylim(0,0.4)
    sca(ax[0,1])
    I1_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,0.4)
    
    sca(ax[1,0])
    ff = R_tot
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[1,1])
    ff = R_tot
    plot(wl, ff ,'-')
    xlim(zoom)
    xlabel('wavelength (nm)')
    grid(True)
    
    sca(ax[2,0])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, RRS in)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[2,1])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    xlim(zoom)
    grid(True)

    sca(ax[3,0])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylabel('$Ring spectrum (Tot/Elastic)$')
    ylim(0.95,1.05)
    grid(True)
    sca(ax[3,1])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylim(0.95,1.05)
    grid(True)
    xlim(zoom)
    
    sca(ax[4,0])
    DoLP_LUT.plot(label='%d'%i)
    ylim(0,40)
    sca(ax[4,1])
    DoLP_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,40)
legend()

In [ ]:
#fig.tight_layout()
fig.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C2_case5_NO2_spectre.png', dpi=200)

### C3 cloud

In [ ]:
# C3 IPRT Cumulus data
cumulus_f='/home/did/RTC/SMART-G/smartg/validation/IPRT/cumulus.dat'
f       = open(cumulus_f, 'r')
f.readline(); 
l       = f.readline().split()
Nx, Ny, Nz, _ = list(map(int, l)) # number of cumulus cells in X, Y, X
Nl      = Nz + 1                  # number of levels in Z
l       = f.readline()
g_info  = list(map(float, l.split()))
Dx, Dy  = g_info[0:2]              # horizontal grid interval in km
zgrid   = np.array(g_info[2:])# vertical grid in km
assert Nl==zgrid.size

zgrid = np.concatenate([zgrid, np.array([120.])])
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Dx=Dx, Dy=Dy, z=zgrid, SAT_ALTITUDE=120, periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ
znew  = zgrid[::-1]

In [ ]:
%%time
# absorption (only gas)
aco2_int = get_co2_abs(znew, wl, DOWNLOAD=False, verbose=True)

In [ ]:
semilogy(wl,aco2_int[:,-1])

In [ ]:
## scattering optical properties
wl_ref  = (wls[0]+wls[-1])/2.
# cloud
reff = 5.  # mic.
cumulus     = np.loadtxt(cumulus_f, skiprows=3, dtype=np.float32)
sca_cld_in  = cumulus[:,3]            # km-1
#sca_cld_in  = cumulus[:2000,3]            # km-1
cloud_sub   = cloud_LUT.sub({'effective_radius':Idx(reff),'effective_variance':0})
eps_cld     = cloud_sub['normed_ext_coeff' ][Idx(wls, fill_value='extrema')]/\
              cloud_sub['normed_ext_coeff' ][Idx(wl_ref, fill_value='extrema')]
sca_cld     = sca_cld_in[None, :] * eps_cld[:, None]

cloud_sub   = cloud_LUT.sub({'effective_radius':Idx(reff),'effective_variance':0, 
                             'wavelength':Idx(wl_ref*1e-3)})
pha         = np.stack([(cloud_sub['p11_phase_function']+cloud_sub['p21_phase_function'])[::-1],
                (cloud_sub['p11_phase_function']-cloud_sub['p21_phase_function'])[::-1],
                 cloud_sub['p34_phase_function'][::-1],
                 cloud_sub['p44_phase_function'][::-1]])
#pha_cld     = LUT(pha, axes=[None, cloud_sub.axis('phase_angles')[::-1]], names=['stk', 'theta_atm'])
#alternative reff = 2 micron phase function
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl_ref)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl_ref)['phase_atm'].sub()[0,:,:]

#Rayleigh
sca_ray     = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(wls), 'OD_r')

# indexing of C3 cells into SMART-G grid
cloud_geo   = cumulus[:,:3].astype(int32)-1
#cloud_geo   = cumulus[:2000,:3].astype(int32)-1
cloud_cell  = np.ravel_multi_index((cloud_geo[:,0], cloud_geo[:,1], cloud_geo[:,2]),
                    dims=(NX, NY, NZ))

# Concatenation
sca_cld3D   = np.concatenate([np.zeros_like(sca_ray), sca_cld], axis=1)
sca_ray3D   = np.concatenate([sca_ray, sca_ray[:,NZ-idz[cloud_cell]]], axis=1)
ssa_cld3D   = np.ones_like(sca_cld3D)
ipha3D      = np.zeros_like(sca_cld3D, dtype=int32)

## Mapping optical propoerties and cells
# 1 clear profile + number of cloud cells opt. prop. (here 1 per cell)
Nopt        = NZ + 1 + cloud_cell.size
iopt        = np.zeros(Ncell, dtype=int32)
iabs        = np.zeros_like(iopt)
iopt[:]     = np.arange(Nopt)[NZ-idz] # Scattering depending on Z for clear atmosphere (Rayleigh)
iabs[:]     = np.arange(Nopt)[NZ-idz] # Absorption depending on Z only
iopt[cloud_cell] = NZ + 1 + np.arange(cloud_cell.size)  # Scattering depending on cloud cell

atm_arr = np.zeros((Nopt,9))
atm_arr[:,0] = np.arange(Nopt)[::-1]
np.savetxt('./tmp.dat', atm_arr)
Nabs        = iabs.max().astype(np.int32)

In [ ]:
atm3D = AtmAFGL('./tmp.dat', US=False,
                    grid        = np.arange(Nopt),
                    prof_ray    = sca_ray3D,
                    prof_aer    = (sca_cld3D, ssa_cld3D),
                    prof_phases = (ipha3D, [pha_cld]),
                    cells       = (iopt, iabs, pmin, pmax, neigh)
                   )
pro3D  = atm3D.calc(wls)

In [ ]:
# Sun position
SAA    = 180.
SZA    = 25.
# Sensors position
Npix   = 100
POSZ   = 5
THDEG  = 180.
PHDEG  = 180.
# pixels centers on the ground
x0     = np.linspace(-6.67/2., 6.67/2, num=Npix) # central domain (cumulus) boundaries
y0     = np.linspace(-6.67/2., 6.67/2, num=Npix)
#
xx,yy  = np.meshgrid(x0, y0)
zz     = np.zeros_like(xx) + POSZ

In [ ]:
%%time
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid,zgrid, xx.ravel(), yy.ravel(), zz.ravel())
#icells_old = locate_3D_cells(pro3D_cld, xx.ravel(), yy.ravel(), zz.ravel())
sensors=[]
for POSX,POSY,ICELL in zip(xx.ravel(), yy.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

S3D     = Smartg(opt3D=True, alt_pp=True, alis=False, back=True, verbose_photon=False ) # 3D
S3DA    = Smartg(opt3D=True, alt_pp=True, alis=True, back=True) # 3D
le      = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
ALB     = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.grass.green.solid.gras.spectrum.txt')
env     = Environment(ENV=1, ALB=ALB, ENV_SIZE=1e4, X0=1e4)
#env     = None

In [ ]:
%%time
NB     = len(sensors) * 1e3
surf   = RoughSurface(WIND=5)
#surf   = LambSurface(ALB=0.2)
m3D    = S3D.run(wl=wls,  NBPHOTONS=NB, atm=pro3D, sensor=sensors, le=le, surf=surf, NBLOOP=1e5, env=env,
               stdev=False, NF=1e4, SEED=-1)

In [ ]:
f=figure(figsize=(8,8))
print(NB)
imshow(m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:],vmax=1, cmap=cm.RdBu_r)
cbar = colorbar(shrink=0.8, orientation='vertical')
cbar.ax.set_xlabel(mdesc('I_up (TOA)'))
f.tight_layout()
#f.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C3_case5_image.png', dpi=200)
f=figure(figsize=(8,8))
I=m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
Q=m3D['Q_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
U=m3D['U_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
V=m3D['V_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
imshow(np.sqrt(Q*Q+U*U+V*V)/I*100,cmap=cm.RdBu_r)
cbar = colorbar(shrink=0.8, orientation='vertical')
cbar.ax.set_xlabel(mdesc('DoLP_up (TOA)'))
f.tight_layout()

In [ ]:
%%time
II      = m3D['I_up (TOA)'].sub()[:,0,0].data
shadows = np.where( II < 0.1)[0]
clouds  = np.where( II > 0.6)[0]
grounds = np.where( (II > 0.15) & (II < 0.3))[0]
sensor_shad = sensors[shadows[10]]
sensor_cld1 = sensors[clouds[10]]
sensor_grd  = sensors[grounds[10]]
# Spectra
#zoom          = (686.25, 687.25)
#zoom          = (691, 692)
zoom          = (2004,2006)
sigma         = aco2_int[:,1:]
#sigma[:]=0
NWS           = wls.size
#sigma         = gas_abs[:,1:]
le            = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
ALB           = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.trees.deciduous.solid.decidou.spectrum.txt')
fig,ax        = subplots(nrows=5, ncols=2) 
fig.set_size_inches(24,15)
surf          = LambSurface(ALB=1.)
surfref       = LambSurface(ALB=ALB.get(wls[0]))
NL            = Nabs # number of abs properties
NB            = 1e4

#for i,sensor in enumerate([sensor_grd]):
for i,sensor in enumerate([sensor_cld1, sensor_grd, sensor_shad]):
    m       = S3DA.run(wl=wls,  NBPHOTONS=NB, atm=pro3D, sensor=sensor, le=le, surf=surf, NBLOOP=1e4,
               stdev=False, NF=1e4, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0})
    #mref    = S3D.run(wl=wls[0],  NBPHOTONS=NB*100, atm=pro3D0, sensor=sensor, le=le, surf=surfref, NBLOOP=1e5,
    #           stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

    w     = m['disth_up (TOA)'][:, NL+4:-3]
    ngood = np.sum(w[:,0]!=0)

    S       = np.zeros((ngood,4),dtype=np.float32) 
    cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
    S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
    w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
    nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
    nref    = m['disth_up (TOA)'][:ngood,      -2  ]
    nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

    XBLOCK  = 512
    XGRID   = 512
    NT      = XBLOCK*XGRID       # Maximum Number of threads
    NPHOTON = cd.shape[0]        # Number of photons

    NLAYER  = sigma.shape[1]     # Number of vertical layer
    NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
    NGROUP  = NT//NWVL           # Number of groups of photons
    NTHREAD = NGROUP*NWVL        # Number of threads used
    NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
    NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
    NWVL_LOW = NWS

    print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
    sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
    sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

    alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32) 

    cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
    weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
    nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
    nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
    nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

    res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
    reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                          np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                          to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                          to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                          block=(XBLOCK,1,1),grid=(XGRID,1,1))
    I1      = res_out.get()/m[0][0,0]
    #norm1   = I1[0,0]/mref['I_up (TOA)'].data
    P_Raman = res_rrs.get()[0,:]/res_out.get()[0,:]
    #print(norm1)
    norm1 = 1
    I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))
    Q1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('Q_up (TOA)'))
    U1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('U_up (TOA)'))
    DoLP_LUT = ((Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)/I1_LUT)*100
    DoLP_LUT.desc='DoLP (%)'
    P_Raman_LUT = LUT(P_Raman, axes=[wl], names=['wavelength'], desc=r'$P_{Raman}$')
    Rel     = Es * I1_LUT[Idx(wl,fill_value='extrema')]
    Rel_llp = Es_RRS * I1_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Rel_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rel_out = Rel * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    R_tot   = R_RRS + Rel - Rel_out
    
    sca(ax[0,0])
    I1_LUT.plot(label='%d'%i)
    ylim(0,0.8)
    sca(ax[0,1])
    I1_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,0.8)
    
    sca(ax[1,0])
    ff = R_tot
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[1,1])
    ff = R_tot
    plot(wl, ff ,'-')
    xlim(zoom)
    xlabel('wavelength (nm)')
    grid(True)
    
    sca(ax[2,0])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, RRS in)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[2,1])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    xlim(zoom)
    grid(True)

    sca(ax[3,0])
    ff = R_tot/Rel
    plot(wl, ff)
    xlabel('wavelength (nm)')
    ylabel('$Ring spectrum (Tot/Elastic)$')
    ylim(0.98,1.05)
    grid(True)
    sca(ax[3,1])
    ff = R_tot/Rel
    if i==0 : plot(wl, ff, '+-')
    else : plot(wl, ff)
    xlabel('wavelength (nm)')
    ylim(0.98,1.05)
    grid(True)
    xlim(zoom)
    
    sca(ax[4,0])
    DoLP_LUT.plot(label='%d'%i)
    ylim(0,40)
    sca(ax[4,1])
    DoLP_LUT.plot(label='%d'%i)
    xlim(zoom)
    ylim(0,40)
legend()

In [ ]:
w     = m['disth_up (TOA)'][:, NL+4:-3]
plot(w[:ngood,0])
print(w[:ngood,0].mean())
figure()
plot(w[:ngood,1])
print(w[:ngood,1].mean())
figure()
plot(w[:ngood,2])
print(w[:ngood,2].mean())

In [ ]:
imshow(m3D['I_up (TOA)'][:,0:3,0,0].reshape(100,100,3))

# Trattoria 2020

In [ ]:
S3D    = Smartg(opt3D=True, alt_pp=True, alis=False, back=True) # 3D
S3DA   = Smartg(opt3D=True, alt_pp=True, alis=True,  back=True) # 3D

In [ ]:
cloud_LUT = read_mlut('/home/did/RTC/SMART-G/smartg/data/opt_liquid_cloud.nc')

In [ ]:
#Solar Spectrum raw data
#datas = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_full.dat')
datas = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_0.1nm.dat')

## 3D Grid & Scene

In [ ]:
# C3 IPRT Cumulus data
cumulus_f='/home/did/RTC/SMART-G/smartg/validation/IPRT/cumulus.dat'
f       = open(cumulus_f, 'r')
f.readline(); 
l       = f.readline().split()
Nx, Ny, Nz, _ = list(map(int, l)) # number of cumulus cells in X, Y, X
Nl      = Nz + 1                  # number of levels in Z
l       = f.readline()
g_info  = list(map(float, l.split()))
Dx, Dy  = g_info[0:2]            # horizontal grid interval in km
#Dx*=10. ; Dy*=10.
zgrid   = np.array(g_info[2:])     # vertical grid in km
assert Nl==zgrid.size

zgrid = np.concatenate([zgrid, np.array([120.])])
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Dx=Dx, Dy=Dy, z=zgrid, SAT_ALTITUDE=120, periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ
znew  = zgrid[::-1]

In [ ]:
# Sun position
SAA    = 180.
SZA    = 40.
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
# Sensors position
Npix   = 100
POSZ   = 30.
THDEG  = 180.
PHDEG  = 180.
# pixels centers on the ground
x0     = np.linspace(-Nx*Dx/2., Nx*Dx/2, num=Npix) # central domain (cumulus) boundaries
y0     = np.linspace(-Ny*Dy/2., Ny*Dy/2, num=Npix)
#x0     = np.linspace(-6.67/2., 6.67/2, num=Npix) # central domain (cumulus) boundaries
#y0     = np.linspace(-6.67/2., 6.67/2, num=Npix)
#
xx,yy  = np.meshgrid(x0, y0)
zz     = np.zeros_like(xx) + POSZ

# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid,zgrid, xx.ravel(), yy.ravel(), zz.ravel())
# sensors objects
sensors=[]
for POSX,POSY,ICELL in zip(xx.ravel(), yy.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

# Ground
ALB     = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.grass.green.solid.gras.spectrum.txt')
env     = Environment(ENV=1, ALB=ALB, ENV_SIZE=10, X0=1e1)
env     = None
surf    = RoughSurface(WIND=5)
surf    = LambSurface(ALB=Albedo_cst(ALB.get(wls[0])))

## Spectral range & gas absorption

In [ ]:
%%time
pstd = np.array([1000,950,900,850,800,700,600,500,400,300,\
            200,125,50,10,3,1], dtype=np.float32)
zstd = ( -np.log(pstd/pstd[0])*8)[::-1]

wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(758., 770.,   datas, dl=0.002, dls=10.)
o2_int  = get_o2_abs (zstd, wl, DOWNLOAD=False, verbose=False, zint=znew)
abs_int = o2_int

#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(1990., 2050., datas, dl=0.002, dls=10.)
#co2_int = get_co2_abs(zstd, wl, DOWNLOAD=False, verbose=True, zint=znew)
#abs_int = co2_int

#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(310., 320., datas, dl=0.01, dls=5.)
#no2_int = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(wl), 'OD_g')
#abs_int = no2_int

#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(530., 550., datas, dl=0.002, dls=5.)
#o3_int = od2k(AtmAFGL('afglt', O3=300., NO2=True, grid=znew).calc(wl), 'OD_g')
#abs_int = o3_int

# Solar spectrum
Es = Es_LUT[Idx(wl)]
Es_RRS = Es_LUT[Idx(wl_RRS, fill_value='extrema')]
# Ring nromalized spectrum (Wagner et al., 2009, AMT)
_, LRRS = L2d_inv(wl, 90., 243.)
f_RRS_norm  = np.sum(Es_RRS * LRRS , axis=1) / Es - 1.

## Scattering optical properties

In [ ]:
wl_ref  = (wls[0]+wls[-1])/2.
# cloud
reff = 5.  # mic.
cumulus     = np.loadtxt(cumulus_f, skiprows=3, dtype=np.float32)
sca_cld_in  = cumulus[:,3]            # km-1
cloud_sub   = cloud_LUT.sub({'effective_radius':Idx(reff),'effective_variance':0})
eps_cld     = cloud_sub['normed_ext_coeff' ][Idx(wls, fill_value='extrema')]/\
              cloud_sub['normed_ext_coeff' ][Idx(wl_ref, fill_value='extrema')]
sca_cld     = sca_cld_in[None, :] * eps_cld[:, None]

cloud_sub   = cloud_LUT.sub({'effective_radius':Idx(reff),'effective_variance':0, 
                             'wavelength':Idx(wl_ref*1e-3)})
pha         = np.stack([(cloud_sub['p11_phase_function']+cloud_sub['p21_phase_function'])[::-1],
                (cloud_sub['p11_phase_function']-cloud_sub['p21_phase_function'])[::-1],
                 cloud_sub['p34_phase_function'][::-1],
                 cloud_sub['p44_phase_function'][::-1]])
#pha_cld     = LUT(pha, axes=[None, cloud_sub.axis('phase_angles')[::-1]], names=['stk', 'theta_atm'])
#alternative reff = 2 micron phase function
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl_ref)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl_ref)['phase_atm'].sub()[0,:,:]

#Rayleigh
sca_ray     = od2k(AtmAFGL('afglt', O3=0., NO2=True, grid=znew).calc(wls), 'OD_r')

# indexing of C3 cells into SMART-G grid
cloud_geo   = cumulus[:,:3].astype(int32)-1
cloud_cell  = np.ravel_multi_index((cloud_geo[:,0], cloud_geo[:,1], cloud_geo[:,2]),
                    dims=(NX, NY, NZ))

# Concatenation
sca_cld3D   = np.concatenate([np.zeros_like(sca_ray), sca_cld], axis=1)
sca_ray3D   = np.concatenate([sca_ray, sca_ray[:,NZ-idz[cloud_cell]]], axis=1)
#sca_ray3D2   = np.concatenate([sca_ray2, sca_ray2[:,NZ-idz[cloud_cell]]], axis=1)
ssa_cld3D   = np.ones_like(sca_cld3D)
ipha3D      = np.zeros_like(sca_cld3D, dtype=int32)

## Mapping optical propoerties and cells
# 1 clear profile + number of cloud cells opt. prop. (here 1 per cell)
Nopt        = NZ + 1 + cloud_cell.size
iopt        = np.zeros(Ncell, dtype=int32)
iabs        = np.zeros_like(iopt)
iopt[:]     = np.arange(Nopt)[NZ-idz] # Scattering depending on Z for clear atmosphere (Rayleigh)
iabs[:]     = np.arange(Nopt)[NZ-idz] # Absorption depending on Z only
iopt[cloud_cell] = NZ + 1 + np.arange(cloud_cell.size)  # Scattering depending on cloud cell

atm_arr = np.zeros((Nopt,9))
atm_arr[:,0] = np.arange(Nopt)[::-1]
np.savetxt('./tmp.dat', atm_arr)
Nabs        = iabs.max().astype(np.int32)

### profiles computations
atm3D = AtmAFGL('./tmp.dat', US=False,
                    grid        = np.arange(Nopt),
                    prof_ray    = sca_ray3D,
                    prof_aer    = (sca_cld3D, ssa_cld3D),
                    prof_phases = (ipha3D, [pha_cld]),
                    cells       = (iopt, iabs, pmin, pmax, neigh)
                   )
pro3D  = atm3D.calc(wls)

## Image

In [ ]:
%%time
NB     = len(sensors) * 1e3
m3D    = S3D.run(wl=wls[0:2], NBPHOTONS=NB, NBLOOP=1e5, atm=pro3D.sub({'wavelength':np.arange(2)}), 
                 sensor=sensors, le=le, surf=surf, env=env, NF=1e4)

In [ ]:
rcParams.update({'font.size':18})
f,ax = subplots(1,1)
f.set_size_inches(8,8)
II   = m3D['I_up (TOA)'][:,0,0,0].reshape(Npix,Npix)
img  = ax.imshow(II[-1:1:-1,1:-1:1], vmax=1, cmap=cm.Blues_r,
       extent=[x0.min(),x0.max(),y0.min(),y0.max()])
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
cbar = f.colorbar(img, shrink=0.7, orientation='vertical')
cbar.ax.set_xlabel(mdesc('I_up (TOA)'))

## POI
shadows = np.where( (II.ravel() > 0.005) & (II.ravel() < 0.015))[0]
clouds  = np.where( II.ravel() > 0.8)[0]
#waters  = np.where( (II.ravel() > 0.02) & (II.ravel() < 0.03))[0]
lands   = np.where( (II.ravel() > 0.07) & (II.ravel() < 0.15))[0]
#sensor_shad = sensors[shadows[0]]
#sensor_shad = sensors[shadows[100]]
sensor_cld1 = sensors[clouds[20]]
sensor_land = sensors[lands[150]]
#sensor_wate = sensors[waters[200]]
#ax.scatter(sensor_wate.dict['POSY'], sensor_wate.dict['POSX'], color='c', marker='*')
ax.scatter(sensor_cld1.dict['POSX'], sensor_cld1.dict['POSY'], color='r', marker='*')
ax.scatter(sensor_land.dict['POSX'], sensor_land.dict['POSY'], color='k', marker='*')
#ax.scatter(sensor_shad.dict['POSX'], sensor_shad.dict['POSY'], color='g', marker='*')
f.tight_layout()
#f.savefig('/home/did/RTC/SMART-G/smartg/validation/IPRT/C3_case5_image_Trattoria_O2B.png', dpi=200)
#f.savefig('/home/documents/CNES/Trattoria_2020/C3_case5_image_Trattoria_O2B.png', dpi=300)
#f.savefig('/home/documents/Copernicus/CERTO/KO_3D.png', dpi=300)

## Spectra

In [ ]:
%%time
rcParams.update({'font.size':14})
color         = ['r','k','g']
lab           = ['cloud','land','shadow']
#zoom          = (686.25, 687.25)
#zoom          = (691, 692)
#zoom          = (2004,2006)
zoom          = (546,547)
sigma         = abs_int[:,1:]
NWS           = wls.size
fig,ax        = subplots(nrows=4, ncols=2) 
fig.set_size_inches(24,15)
surf          = LambSurface(ALB=Albedo_cst(1))
surfref       = LambSurface(ALB=Albedo_cst(ALB.get(wls[0])))
NL            = Nabs # number of abs properties
NB            = 1e5

for i,sensor in enumerate([sensor_land]):
#for i,sensor in enumerate([sensor_cld1, sensor_land, sensor_shad]):
    m       = S3DA.run(wl=wls,  NBPHOTONS=NB, NBLOOP=1e4, atm=pro3D, sensor=sensor, le=le, surf=surf,
               NF=1e4, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0})
    mref    = S3D.run(wl=wls[0:2],  NBPHOTONS=NB*10, atm=pro3D.sub({'wavelength':np.arange(2)}), sensor=sensor, 
                le=le, surf=surfref, NBLOOP=1e5, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0})

    w     = m['disth_up (TOA)'][:, NL+4:-3]
    ngood = np.sum(w[:,0]!=0)

    S       = np.zeros((ngood,4),dtype=np.float32) 
    cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
    S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
    w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
    nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
    nref    = m['disth_up (TOA)'][:ngood,      -2  ]
    nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

    XBLOCK  = 512
    XGRID   = 512
    NT      = XBLOCK*XGRID       # Maximum Number of threads
    NPHOTON = cd.shape[0]        # Number of photons

    NLAYER  = sigma.shape[1]     # Number of vertical layer
    NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
    NGROUP  = NT//NWVL           # Number of groups of photons
    NTHREAD = NGROUP*NWVL        # Number of threads used
    NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
    NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
    NWVL_LOW = NWS

    print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
    sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
    sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

    alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32) 

    cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
    weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
    nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
    nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
    nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

    res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
    reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                          np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                          to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                          to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                          block=(XBLOCK,1,1),grid=(XGRID,1,1))
    I1      = res_out.get()/m[0][0,0]
    norm1   = I1[0,0]/mref['I_up (TOA)'][0]
    P_Raman = res_rrs.get()[0,:]/res_out.get()[0,:]
    print(norm1)
    #norm1 = 1
    I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))
    Q1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('Q_up (TOA)'))
    U1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('U_up (TOA)'))
    DoLP_LUT = ((Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)/I1_LUT)*100
    DoLP_LUT.desc='DoLP (%)'
    P_Raman_LUT = LUT(P_Raman, axes=[wl], names=['wavelength'], desc=r'$P_{Raman}$')
    Rel     = Es * I1_LUT[Idx(wl,fill_value='extrema')]
    Rel_llp = Es_RRS * I1_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Rel_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rel_out = Rel * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    R_tot   = R_RRS + Rel - Rel_out
    
    sca(ax[0,0])
    I1_LUT.plot(label=lab[i], fmt=color[i])
    ylim(0,0.8)
    sca(ax[0,1])
    I1_LUT.plot(label=lab[i], fmt=color[i])
    xlim(zoom)
    ylim(0,0.8)
    legend()
    
    sca(ax[1,0])
    ff = R_tot
    plot(wl, ff ,'-'+color[i])
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[1,1])
    ff = R_tot
    if i==0 : plot(wl, ff, '+-'+color[i])
    else : plot(wl, ff, color[i])
    xlim(zoom)
    xlabel('wavelength (nm)')
    grid(True)
    
    '''sca(ax[2,0])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    ylabel(mdesc('I_up (TOA, RRS in)')+ r'$ (ph/s/cm^2/nm)$')
    grid(True)
    sca(ax[2,1])
    ff = R_RRS
    plot(wl, ff ,'-')
    xlabel('wavelength (nm)')
    xlim(zoom)
    grid(True)'''

    sca(ax[2,0])
    ff = R_tot/Rel
    plot(wl, ff, '-'+color[i])
    xlabel('wavelength (nm)')
    ylabel('$Ring spectrum (Tot/Elastic)$')
    ylim(0.98,1.05)
    grid(True)
    sca(ax[2,1])
    ff = R_tot/Rel
    plot(wl, ff ,'-'+color[i])
    xlabel('wavelength (nm)')
    ylim(0.98,1.05)
    grid(True)
    xlim(zoom)
    
    sca(ax[3,0])
    DoLP_LUT.plot(fmt=color[i])
    ylim(0,20)
    sca(ax[3,1])
    DoLP_LUT.plot(fmt=color[i])
    xlim(zoom)
    ylim(0,20)


# Validation

## 1D RRS vs McArtim

In [ ]:
from scipy.signal import gaussian
from scipy.integrate import simps

def myconvolve(x, N=250):
    srf = gaussian(6*N, N*1.)
    srf/= simps(srf)
    return convolve(x, srf, mode='same')

def reduce_histories(m, wls, wl, sigma):
    
    NWS           = wls.size
    NL            = sigma.shape[1]
    w     = m['disth_up (TOA)'][:, NL+4:-3]
    ngood = np.sum(w[:,0]!=0)

    S       = np.zeros((ngood,4),dtype=np.float32) 
    cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
    S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
    w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
    nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
    nref    = m['disth_up (TOA)'][:ngood,      -2  ]
    nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

    XBLOCK  = 512
    XGRID   = 512
    NT      = XBLOCK*XGRID       # Maximum Number of threads
    NPHOTON = cd.shape[0]        # Number of photons

    NLAYER  = sigma.shape[1]     # Number of vertical layer
    NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
    NGROUP  = NT//NWVL           # Number of groups of photons
    NTHREAD = NGROUP*NWVL        # Number of threads used
    NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
    NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
    NWVL_LOW = NWS

    print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
    sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
    sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

    #alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32)
    alb_in       = np.zeros(NWVL, dtype=np.float32)

    cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
    S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
    weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
    nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
    nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
    nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

    res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
    res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
    reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                          np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                          to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                          to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                          block=(XBLOCK,1,1),grid=(XGRID,1,1))
    I1      = res_out.get()/m[0][0,0]
    #norm1   = I1[0,0]/mref['I_up (TOA)'][0]
    norm1=1.
    P_Raman = res_rrs.get()[0,:]/res_out.get()[0,:]
    print(norm1)
    #norm1 = 1
    I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))
    Q1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('Q_up (TOA)'))
    U1_LUT = LUT(I1[1,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('U_up (TOA)'))
    DoLP_LUT = ((Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)/I1_LUT)*100
    DoLP_LUT.desc='DoLP (%)'
    Ip_LUT   = (Q1_LUT*Q1_LUT + U1_LUT*U1_LUT).apply(np.sqrt)
    Ip_LUT.desc=mdesc('Ip_up (TOA)')
    P_Raman_LUT = LUT(P_Raman, axes=[wl], names=['wavelength'], desc=r'$P_{Raman}$')
    Rel     = Es * I1_LUT[Idx(wl,fill_value='extrema')]
    Rel_llp = Es_RRS * I1_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Rel_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rel_out = Rel * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    R_tot   = R_RRS + Rel - Rel_out
    #------------------------
    Relp    = Es * Ip_LUT[Idx(wl,fill_value='extrema')]
    Relp_llp= Es_RRS * Ip_LUT[Idx(wl_RRS,fill_value='extrema')]
    R_RRS   = np.sum(Relp_llp * LRRS, axis=1) * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Relp_out= Relp * P_Raman_LUT[Idx(wl, fill_value='extrema')]
    Rp_tot  = R_RRS + Relp - Relp_out
    
    return R_tot, Rel, DoLP_LUT, Rp_tot, Relp

#datas = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_full.dat')
#datas_01 = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_0.1nm.dat')
datas_modtran = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/atlas_plus_modtran_ph')

In [ ]:
# pure molecular atmosphere (fig 7.5 and 7.6 Deutschmann PhD thesis)
pstd = np.array([1000,950,900,850,800,700,600,500,400,300,\
            200,125,50,10,3,1], dtype=np.float32)
zstd = ( -np.log(pstd/pstd[0])*8)[::-1]

atm  = AtmAFGL('afglt', O3=300., NO2=True, grid=zstd)
le   = {'phi_deg':np.linspace(0.,180., num=1), 'th_deg':np.linspace(0., 85., num=1)}
th0  = 45.
#mydatas = np.zeros_like(datas)
#mydatas[:,1] = myconvolve(datas[:,1], N=25)
#mydatas[:,0] = datas[:,0]

In [ ]:
%%time
wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(758., 770.,   datas_modtran, dl=0.001, dls=1.)
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(758., 770.,  datas, dls=1.)
o2_int  = get_o2_abs (zstd, wl, DOWNLOAD=False)
abs_int = o2_int

#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(352., 360., mydatas, dls=5.)
#o3_int = od2k(AtmAFGL('afglt', O3=0., NO2=False, grid=zstd).calc(wl), 'OD_g')
#abs_int = o3_int

# Solar spectrum
Es = Es_LUT[Idx(wl)]
Es_RRS = Es_LUT[Idx(wl_RRS, fill_value='extrema')]
# Ring nromalized spectrum (Wagner et al., 2009, AMT)
_, LRRS = L2d_inv(wl, 90., 243.)
f_RRS_norm  = np.sum(Es_RRS * LRRS , axis=1) / Es - 1.

In [ ]:
%%time
fig,ax        = subplots(nrows=3, ncols=1) 
fig.set_size_inches(20,15)

rcParams.update({'font.size':14})
sigma         = abs_int[:,1:]

color         = ['b']
lab           = ['smartg']
i=0
surf          = LambSurface(ALB=Albedo_cst(1))
#surf          = None
NB            = 1e5
NWS           = wls.size

m             = Smartg(double=True, alt_pp=True, alis=True).run(wl=wls, THVDEG=th0,
                       NBPHOTONS=NB, NBLOOP=NB, atm=atm, le=le, surf=surf,
                       NF=1e3, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0}) 
    
R_tot, Rel, DoLP_LUT, Rp_tot, Relp = reduce_histories(m,wls,wl,sigma)
###################################################################################

sca(ax[0])
ff = R_tot
plot(wl, ff ,'-'+color[i])
ff = Rp_tot
plot(wl, ff ,':'+color[i])
xlabel('wavelength (nm)')
ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
grid(True)

sca(ax[1])
ff = (R_tot/Rel-1)*100
plot(wl, ff, '-'+color[i])
ff = (Rp_tot/Relp-1)*100
plot(wl, ff, ':'+color[i])
xlabel('wavelength (nm)')
ylabel('$Ring spectrum (Tot/Elastic)$')
#plot(wl, myconvolve(ff, N=35), '-m')
#ylim(-20,30)
ylim(-6,8)
grid(True)

sca(ax[2])
DoLP_LUT.plot(fmt=color[i])
#ylim(0,2)

In [ ]:
# pure molecular atmosphere (fig 7.7 Deutschmann PhD thesis)
pstd = np.array([1000,950,900,850,800,700,600,500,400,300,\
            200,125,50,10,3,1], dtype=np.float32)
zstd = ( -np.log(pstd/pstd[0])*8)[::-1]

atm  = AtmAFGL('afglus', O3=300., NO2=False)
#atm  = AtmAFGL('afglus', O3=300., NO2=False, grid=zstd)
le   = {'phi_deg':np.linspace(0.,180., num=1), 'th_deg':np.linspace(0., 85., num=1)}
th0  = 0.
#mydatas = np.zeros_like(datas)
#mydatas[:,1] = myconvolve(datas[:,1], N=35)
#mydatas[:,0] = datas[:,0]

In [ ]:
%%time
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(758., 770.,   datas, dl=0.001, dls=5.)
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(758., 770.,  mydatas, dls=5.)
#o2_int  = get_o2_abs (zstd, wl, DOWNLOAD=False)
#abs_int = o2_int

wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(351., 354., datas_modtran, dls=1.)
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(351., 354., mydatas, dls=5.)
o3_int = od2k(atm.calc(wl), 'OD_g')
abs_int = o3_int

# Solar spectrum
Es = Es_LUT[Idx(wl)]
Es_RRS = Es_LUT[Idx(wl_RRS, fill_value='extrema')]
# Ring nromalized spectrum (Wagner et al., 2009, AMT)
_, LRRS = L2d_inv(wl, 90., 243.)
f_RRS_norm  = np.sum(Es_RRS * LRRS , axis=1) / Es - 1.

In [ ]:
uvspec_tot = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/examples/UVSPEC_RAMAN.OUT')[:,1]
uvspec_el  = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/examples/UVSPEC_NORAMAN.OUT')[:,1]
ff_uvspec = (uvspec_tot/uvspec_el-1)*100

In [ ]:
%%time
fig,ax        = subplots(nrows=3, ncols=1) 
fig.set_size_inches(20,15)

rcParams.update({'font.size':14})
sigma         = abs_int[:,1:]

color         = ['b']
lab           = ['smartg']
i=0
surf          = LambSurface(ALB=0.)
surf          = None
NB            = 1e5
NWS           = wls.size

m             = Smartg(double=True, alt_pp=True, alis=True).run(wl=wls, THVDEG=th0,
                       NBPHOTONS=NB, NBLOOP=NB, atm=atm, le=le, surf=surf,
                       NF=1e3, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0}) 
    
R_tot, Rel, DoLP_LUT = reduce_histories(m,wls,wl,sigma)
###################################################################################

sca(ax[0])
ff = R_tot/(1e-3 * 1e-4  / (cst.h *cst.c)  * (wl*1e-9))*cos(np.deg2rad(th0))/np.pi*1e-3
ffe = Rel/(1e-3 * 1e-4  / (cst.h *cst.c)  * (wl*1e-9))*cos(np.deg2rad(th0))/np.pi*1e-3
plot(wl, ff ,'-'+color[i])
plot(wl, ffe ,'--'+color[i])
#ylim(0,.1)
xlabel('wavelength (nm)')
ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
xlim(wl.min(),wl.max())
grid(True)

sca(ax[1])
ff = (R_tot/Rel-1)*100
plot(wl, ff, '-'+color[i])
plot(wl, ff_uvspec, ':'+'r')
xlabel('wavelength (nm)')
ylabel('$Ring spectrum (Tot/Elastic)$')
#plot(wl, myconvolve(ff, N=35), '-m')
ylim(-5,5)
grid(True)
xlim(wl.min(),wl.max())

sca(ax[2])
DoLP_LUT.plot(fmt=color[i])
xlim(wl.min(),wl.max())
#ylim(0,2)

In [ ]:
# pure molecular atmosphere (fig 7.11 Deutschmann PhD thesis)
pstd = np.array([1000,950,900,850,800,700,600,500,400,300,\
            200,125,50,10,3,1], dtype=np.float32)
zstd = ( -np.log(pstd/pstd[0])*8)[::-1]
RTER = 6371.
atm  = AtmAFGL('afglt', O3=0., NO2=False, grid=zstd)
le   = {'phi_deg':np.array([106.25]), 'th_deg':np.array([57.4])}
#sensor = Sensor(POSZ=16.368, THDEG=92.5, LOC='ATMOS')
sensor = Sensor(POSZ=16.368+RTER, THDEG=87)
#mydatas = np.zeros_like(datas)
#mydatas[:,1] = myconvolve(datas[:,1], N=25)
#mydatas[:,0] = datas[:,0]

In [ ]:
%%time
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(758., 770.,   datas, dl=0.001, dls=5.)
#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(758., 770.,  mydatas, dls=5.)
#o2_int  = get_o2_abs (zstd, wl, DOWNLOAD=False)
#abs_int = o2_int

#wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(352., 360., datas_modtran, dls=5.)
wl, wls, wl_RRS, Es_LUT, iwls_in, wwls_in = spectral_grids(252.4, 252.5, datas, dls=0.04)
o3_int = od2k(atm.calc(wl), 'OD_g')
abs_int = o3_int

# Solar spectrum
Es = Es_LUT[Idx(wl)]
Es_RRS = Es_LUT[Idx(wl_RRS, fill_value='extrema')]
# Ring nromalized spectrum (Wagner et al., 2009, AMT)
_, LRRS = L2d_inv(wl, 90., 243.)
f_RRS_norm  = np.sum(Es_RRS * LRRS , axis=1) / Es - 1.

In [ ]:
NWS

In [ ]:
fig,ax        = subplots(nrows=3, ncols=1) 
fig.set_size_inches(20,15)

rcParams.update({'font.size':14})
sigma         = abs_int[:,1:]

color         = ['b']
lab           = ['smartg']
i=0
surf          = LambSurface(ALB=0.)
surf          = None
NB            = 1e5
NWS           = wls.size

m             = Smartg(alt_pp=True, pp=False, alis=True).run(wl=wls,  THVDEG=2.5,
                       NBPHOTONS=NB, NBLOOP=NB, atm=atm, le=le, surf=surf, RTER=RTER,
                       NF=1e3, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0}) 
    
R_tot, Rel, DoLP_LUT = reduce_histories(m,wls,wl,sigma)
###################################################################################

sca(ax[0])
ff = R_tot/(1e-3 * 1e-4  / (cst.h *cst.c)  * (wl*1e-9))*cos(np.deg2rad(th0))/np.pi*1e-3
ffe = Rel/(1e-3 * 1e-4  / (cst.h *cst.c)  * (wl*1e-9))*cos(np.deg2rad(th0))/np.pi*1e-3
plot(wl, ff ,'-'+color[i])
plot(wl, ffe ,':'+color[i])
#ylim(0,3.2e10)
xlabel('wavelength (nm)')
ylabel(mdesc('I_up (TOA, tot)')+ r'$ (ph/s/cm^2/nm)$')
grid(True)

sca(ax[1])
ff = (R_tot/Rel-1)*100
plot(wl, ff, '-'+color[i])
xlabel('wavelength (nm)')
ylabel('$Ring spectrum (Tot/Elastic)$')
#plot(wl, myconvolve(ff, N=100), '-b')
#ylim(-6,10)
grid(True)
xlim(wl.min(),wl.max())

sca(ax[2])
DoLP_LUT.plot(fmt=color[i])
#ylim(0,2)